In [4]:
import os, textwrap
import io

from typing import List, Dict, Optional
from xml.sax.saxutils import escape

from gpts.gpt_assistants import general_assistant
from dotenv import load_dotenv, find_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.core.exceptions import HttpResponseError
from azure.search.documents.models import HybridSearch


from openai import AzureOpenAI, APIConnectionError, OpenAI
from prompts import new_system_finance_prompt

from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer

from prompts4 import finance_calculations, finance_pairs, capital_pairs, stakeholders_pairs, biz_overview_pairs, revenue_pairs, default_gpt_prompt, section4a, section4b, section5, section3, biz_overview_web, stakeholders_web
from pages.design.func_tools import *
from pages.design.formatting import *
from pages.design.func_tools import docx_bytes_to_pdf_bytes
import re, time
 
load_dotenv(find_dotenv(), override=True)

# ---- Config (expects the same envs you already used) ----
SEARCH_ENDPOINT = os.environ["AZURE_SEARCH_ENDPOINT"]
SEARCH_INDEX    = os.environ["AZURE_SEARCH_INDEX"]
SEARCH_KEY      = os.getenv("AZURE_SEARCH_API_KEY")  # omit if using AAD/RBAC
VECTOR_FIELD    = os.getenv("VECTOR_FIELD")
TEXT_FIELD      = os.getenv("TEXT_FIELD")

AOAI_ENDPOINT   = os.environ["AZURE_OPENAI_ENDPOINT"]            # https://<resource>.openai.azure.com
AOAI_API_VER    = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-10-21")
AOAI_DEPLOYMENT = os.environ["AZURE_OPENAI_DEPLOYMENT"]          # e.g., gpt-4o-mini / o3-mini / gpt-5 preview
AOAI_KEY        = os.getenv("AZURE_OPENAI_API_KEY")              # omit if using AAD
OPENAI_API_KEY  = os.getenv("OPENAI_API_KEY")        # required

# ------------------ CODE

class profileAgent():

    """Hybrid (dense+sparse) RAG over Vector Store

    This Agent is responsible for creating Company Profiles. 
    It operates with gpt5.
    It is activated by a call on main rag when it is typed 'Create company profile'
    """

    def __init__(self, company_name, k, max_text_recall_size, max_chars, model, profile_prompt = new_system_finance_prompt, finance_calculations = finance_calculations):
        
        self.company_name = company_name

        self.k = k
        self.max_text_recall_size = max_text_recall_size
        self.model = model
        self.max_chars = max_chars

        self.azure_credentials = AzureKeyCredential(SEARCH_KEY) if SEARCH_KEY else DefaultAzureCredential()
        self.search_client = SearchClient(SEARCH_ENDPOINT, SEARCH_INDEX, credential=self.azure_credentials)

        self.az_openai = AzureOpenAI(azure_endpoint=AOAI_ENDPOINT, api_key=AOAI_KEY, api_version=AOAI_API_VER)
        self.profile_prompt = profile_prompt
        self.web_openai = OpenAI(api_key=OPENAI_API_KEY)

        self.reasoning_effort = "medium"
        self.verbosity = "medium"

        self.finance_calculations = finance_calculations

    def _company_filter(self) -> str:
        v = (self.company_name or "").replace("'", "''").strip()
        return f"company_name eq '{v}'" if v else None
    
    def assemble_bm25_from_llm(self, slots: dict) -> str:
        def q(s: str) -> str:
            # sanitize: remove internal quotes and trim
            s = (s or "").strip().replace('"', ' ')
            return f"\"{s}\"" if s else ""
        groups = []

        # must-have phrases (ANDed)
        for p in slots.get("must_have_phrases", []):
            qp = q(p)
            if qp:
                groups.append(qp)

        # metric / statement synonym groups (ORed within each group)
        for key in ["metric", "statement"]:
            syns = slots.get("synonyms", {}).get(key, []) or slots.get(key, [])
            syns = [q(s) for s in syns if s]
            if syns:
                groups.append("(" + " OR ".join(syns) + ")")

        return " AND ".join(groups) if groups else "\"financial statements\""


    def bm25_creator(self, prompt):

        instruction = (
            "Extract finance search slots for Azure AI Search. "
            "Return strict JSON: {\"metric\":[], \"statement\":[], \"synonyms\":{}, \"must_have_phrases\":[]} "
            "(include IFRS/US GAAP variants)."
        )
        resp = general_assistant(instruction, prompt, OPENAI_API_KEY, 'gpt-4o')

        try:
            slots = getattr(resp, "output_json", None)
            if slots is None:
                import json
                slots = json.loads(resp.output_text)
        except Exception:
            # fallback: minimal anchors from prompt
            slots = {"must_have_phrases": [prompt], "metric": [], "statement": [], "synonyms": {}}
        return self.assemble_bm25_from_llm(slots)

    def _retrieve_hybrid_enhanced(self, query_nl, k: int = 50, top_n = 30, fields=VECTOR_FIELD, max_text_recall_size:int = 800):
        sc = self.search_client
        flt = self._company_filter()
        
        try:
            vq = VectorizableTextQuery(text=query_nl, k=k, fields=VECTOR_FIELD)
            # Prefer vector-only search (integrated vectorization). If your index isn't set up for it, this raises.
            results = sc.search(
                search_text=self.bm25_creator(query_nl), 
                vector_queries=[vq], 
                top=top_n, 
                query_type="semantic",
                query_caption="extractive", 
                hybrid_search=HybridSearch(max_text_recall_size=self.max_text_recall_size),
                query_caption_highlight_enabled=True,
                filter=flt
                )
            mode = "hybrid + semantic"
        except HttpResponseError as e:
            # Fall back to lexical so you still get results while fixing vector config
            results = sc.search(search_text=self.bm25_creator(query_nl), top=k)
            mode = f"lexical (fallback due to: {e.__class__.__name__})"

        hits: List[Dict] = []
        for r in results:
            d = r.copy() if hasattr(r, "copy") else {k2: r[k2] for k2 in r}
            d["score"] = d.get("@search.reranker_score") or d.get("@search.score") or 0.0
            caps = d.get("@search.captions")
            if isinstance(caps, list) and caps:
                d["caption"] = getattr(caps[0], "text", None)
            hits.append(d)

        return mode, hits


    def _build_context(self, hits: List[Dict], text_field: str = TEXT_FIELD, max_chars: int = 20000):
        """Build a compact, numbered context block and also return the selected chunk metadata."""
        lines = []
        total = 0
        selected = []  # <- we'll return this

        for i, h in enumerate(hits, 1):
            title     = h.get("title")
            chunk_id  = h.get("chunk_id")
            full_text = (h.get(text_field) or "")
            if not full_text:
                continue

            preview = textwrap.shorten(full_text, width=700, placeholder=" ...")
            block = f"[{i}] title={title!r} | chunk_id={chunk_id} | score={h.get('score'):.4f}\n{full_text}"

            if total + len(block) > self.max_chars:
                break

            total += len(block)
            lines.append(block)

            # keep rich metadata so you can show or log it later
            selected.append({
                "i": i,
                "title": title,
                "chunk_id": chunk_id,
                "score": h.get("score"),
                "caption": h.get("caption"),
                "preview": preview,
                "text": full_text,  # full chunk text (not shortened)
                # include any other fields you index, if available:
                "metadata_storage_path": h.get("metadata_storage_path"),
                "page_number": h.get("page_number"),
                "doc_type": h.get("doc_type"),
            })

        return "\n\n---\n\n".join(lines), selected

        
    def _generate_pdf(self, text: str) -> bytes:

        buf = io.BytesIO()
        doc = SimpleDocTemplate(buf, pagesize=letter)
        styles = getSampleStyleSheet()
        body = styles["BodyText"]

        story = []
        # Treat double newlines as paragraph breaks; keep single newlines as <br/>
        for para in (text or "").split("\n\n"):
            safe = escape(para).replace("\n", "<br/>")
            story.append(Paragraph(safe if safe.strip() else "&nbsp;", body))
            story.append(Spacer(1, 8))

        doc.build(story)
        buf.seek(0)
        return buf.getvalue()
    
    def _extract_cited_idxs(self, answer: str) -> list[int]:
        # Matches [#1], [#12], etc. (also tolerates stray [1])
        nums = set(int(n) for n in re.findall(r"\[#?(\d+)\]", answer))
        return sorted(nums)

    def _rag_answer(self, rag_nl, question, k: int = 5, temperature: float = 0.2):

        # question = f'CREATE A SECTION OF COMPANY PROFILE USING LAST YEARS OF ANNUAL REPORT PRESENT IN THE CONTEXT FOR {self.company_name}. IF ANY INFORMATION IS NOT FOUND STATE AS n.a. .\n\n THIS IS THE SECTION TO BE BUILT: \n {section7}  \n USE THIS TO GUIDE YOURSELF ON SEMANTIC TERMS AND HOW TO CALCULATE: \n {finance_calculations}'
        
        mode, hits = self._retrieve_hybrid_enhanced(
            # query=rag_q, 
            query_nl=rag_nl,
            k=25
            )
        ctx_text, ctx_items = self._build_context(hits)

        system_msg = self.profile_prompt + (
            "\nWhen you use a fact from the context, add citations like [#1], [#2]."
            "\nOnly rely on the numbered context; if a value is missing, say 'n.a.'."
            f"\nIF ANY INFORMATION IS NOT FOUND STATE AS n.a. .\n\n USE THIS TO GUIDE YOURSELF ON SEMANTIC TERMS AND HOW TO CALCULATE: \n {finance_calculations}"
        )
        user_msg = f"Question:\n{question}\n\nContext snippets (numbered):\n{ctx_text}"

        client = self.az_openai
        messages = [
            {"role": "system", "content": system_msg},
            {"role": "user",   "content": user_msg},
        ]

        # Try streaming first (SSE). Some networks/proxies block streaming; if so, fall back.
        
        resp = client.chat.completions.create(
            model=AOAI_DEPLOYMENT,
            messages=messages,
            reasoning_effort="high"
        )
        answer = resp.choices[0].message.content
        mode_model = "non-streaming (fallback)"

        cited = self._extract_cited_idxs(answer)
        used_chunks = [c for c in ctx_items if c["i"] in cited]

        # return self._generate_pdf(answer)
        return {
            "answer": answer,
            "citations": cited,          # [1, 3, 7]
            "used_chunks": used_chunks,  # detailed dicts for each cited snippet
            "all_chunks": ctx_items,     # everything you sent (optional)
            "mode": mode                 # retrieval mode info (optional)
        }

    def _web_search(self, messages):
        resp = self.web_openai.responses.create(
            model='gpt-5',
            input=messages,
            tools=[{"type": "web_search"}],
            tool_choice="auto",
            # max_output_tokens=self.max_output_tokens,
            reasoning={"effort": self.reasoning_effort},
            text={"verbosity": self.verbosity},
        )
        
        return resp.output_text
    
    def _answer(self, question, ctx_text, k: int = 5, temperature: float = 0.2):

        system_msg = self.profile_prompt + (
            "\nWhen you use a fact from the context, preserve any existing citations like [#1], [#2], [#5, p.41] that are already in the context text."
            "\nOnly rely on the provided context; if a value is missing, say 'n.a.'."
            "\nIMPORTANT: If the formatting instructions request a Sources section, you MUST include it at the end."
            "\nFor the Sources section, list all citation numbers/references that appear in your answer, and describe what document/source each refers to based on information in the context."
        )
        user_msg = f"Question:\n{question}\n\nContext snippets:\n{ctx_text}"

        client = self.az_openai
        messages = [
            {"role": "system", "content": system_msg},
            {"role": "user",   "content": user_msg},
        ]

        # Try streaming first (SSE). Some networks/proxies block streaming; if so, fall back.

        resp = client.chat.completions.create(
            model=AOAI_DEPLOYMENT,
            messages=messages,
            reasoning_effort="high"
        )
        answer = resp.choices[0].message.content

        cited = self._extract_cited_idxs(answer)

        # return self._generate_pdf(answer)
        return {
            "answer": answer,
            "citations": cited,          # [1, 3, 7]
        }   
    
    @staticmethod
    def has_na(text: str) -> bool:
        # match "n.a." or "n/a" (case-insensitive)
        return bool(re.search(r"\b(n\.a\.|n/a)\b", text, flags=re.I))

    def _sections(self, pairs):

        answers = []

        max_extra_na_retries = 1        # try again at most 2 times (total <= 3 calls per item)
        base_delay_seconds = 3.0        # polite delay between attempts


        for q, r in pairs:
            tries = 0
            while True:
                if tries > 0:
                    # small incremental delay before re-trying
                    time.sleep(base_delay_seconds + 0.5 * tries)

                resp = self._rag_answer(rag_nl=r[0], question=q[0])
                answer_text = resp["answer"]

                # stop if good answer OR we've exhausted retries
                if not profileAgent.has_na(answer_text) or tries >= max_extra_na_retries:
                    answers.append(answer_text)
                    break

                # otherwise, try again
                tries += 1

            # optional small gap between different (r,q) items
            time.sleep(5.0)
        
        return answers
    
    def _generate_section(self, section):

        if section == 'GENERATE BUSINESS OVERVIEW':
            # =========== GENERATE BUSINESS OVERVIEW
            biz_overview_pairs_flat = list(zip(biz_overview_pairs[1], biz_overview_pairs[0]))  # [(r, q), (r, q), ...]
            section_built = self._sections(pairs = biz_overview_pairs_flat)

            #getting web search sections
            new_section = f'All instructions applies to the company: {self.company_name}\n\n{biz_overview_web} \n\n Mention in the Beggining of the answer that this is WEBSEARCH SOURCE'
            messages = [
                {"role": "system", "content": default_gpt_prompt},
                {"role": "user",   "content": new_section},
            ]
            resp_web = self._web_search(messages)

            section_built.append(resp_web)

            # Join all context sections - they already contain their own citations
            # Just concatenate them so the model can synthesize
            ctx_text_formatted = "\n\n".join(section_built)

            resp = self._answer(question=biz_overview_mix_formatting, ctx_text=ctx_text_formatted)
            return resp['answer']
        elif section == 'GENERATE KEY STAKEHOLDERS':
        # =========== GENERATE KEY STAKEHOLDERS
            stakeholders_pairs_flat = list(zip(stakeholders_pairs[1], stakeholders_pairs[0]))  # [(r, q), (r, q), ...]
            section_built = self._sections(pairs= stakeholders_pairs_flat)

            #getting web search sections
            new_section = f'All instructions applies to the company: {self.company_name}\n\n{stakeholders_web} \n\n Mention in the Beggining of the answer that this is WEBSEARCH SOURCE'
            messages = [
                {"role": "system", "content": default_gpt_prompt},
                {"role": "user",   "content": new_section},
            ]
            resp_web = self._web_search(messages)

            section_built.append(resp_web)

            # Join all context sections - they already contain their own citations
            # Just concatenate them so the model can synthesize
            ctx_text_formatted = "\n\n".join(section_built)

            resp = self._answer(question=stakeholders_web_mix, ctx_text=section_built)
            return resp['answer']
        elif section == 'GENERATE FINANCIAL HIGHLIGHTS':
            # =========== GENERATE FINANCIAL HIGHLIGHTS
            finance_pairs_flat = list(zip(finance_pairs[1], finance_pairs[0]))  # [(r, q), (r, q), ...]
            section_built = self._sections(pairs=finance_pairs_flat)
            resp = self._answer(question=finance_formatting_2, ctx_text=section_built)
            return resp['answer']
        elif section == 'GENERATE CAPITAL STRUCTURE':
            # =========== GENERATE CAPITAL STRUCTURE
            capital_pairs_flat = list(zip(capital_pairs[1], capital_pairs[0]))  # [(r, q), (r, q), ...]
            section_built = self._sections(pairs= capital_pairs_flat)
            resp = self._answer(question=capital_structure_formatting_2, ctx_text=section_built)
            return resp['answer']
        elif section == 'GENERATE REVENUE SPLIT':
            # =========== GENERATE CAPITAL STRUCTURE
            revenue_pairs_flat = list(zip(revenue_pairs[1], revenue_pairs[0]))  # [(r, q), (r, q), ...]
            section_built = self._sections(pairs= revenue_pairs_flat)
            resp = self._answer(question=section3, ctx_text=section_built)
            return resp['answer']
        elif section == 'GENERATE PRODUCTS SERVICES OVERVIEW':
            # =========== GENERATE CAPITAL STRUCTURE
            new_section = f'All instructions applies to the company: {self.company_name}\n\n{section4a}'
            messages = [
                {"role": "system", "content": default_gpt_prompt},
                {"role": "user",   "content": new_section},
            ]
            resp = self._web_search(messages)
            return resp 
        elif section == 'GENERATE GEO FOOTPRINT':
            # =========== GENERATE CAPITAL STRUCTURE
            new_section = f'All instructions applies to the company: {self.company_name}\n\n{section4b}'
            messages = [
                {"role": "system", "content": default_gpt_prompt},
                {"role": "user",   "content": new_section},
            ]
            resp = self._web_search(messages)
            return resp
        elif section == 'GENERATE DEVELOPMENTS HIGHLIGHTS':
            # =========== GENERATE CAPITAL STRUCTURE
            new_section = f'All instructions applies to the company: {self.company_name}\n\n{section5}'
            messages = [
                {"role": "system", "content": default_gpt_prompt},
                {"role": "user",   "content": new_section},
            ]
            resp = self._web_search(messages)
            return resp


    def generate_company_profile(self):

        # =========== GENERATE BUSINESS OVERVIEW
        biz_overview_pairs_flat = list(zip(biz_overview_pairs[1], biz_overview_pairs[0]))  # [(r, q), (r, q), ...]
        section1 = self._sections(pairs = biz_overview_pairs_flat)
        resp = self._answer(question=business_overview_formatting, ctx_text=section1)
        doc = insert_biz_overview(resp['answer'])

        time.sleep(60)
        # =========== GENERATE KEY STAKEHOLDERS
        stakeholders_pairs_flat = list(zip(stakeholders_pairs[1], stakeholders_pairs[0]))  # [(r, q), (r, q), ...]
        section2 = self._sections(pairs= stakeholders_pairs_flat)
        resp = self._answer(question=stakeholders_formatting, ctx_text=section2)
        doc = insert_stakeholders(resp['answer'], doc=doc)
        
        time.sleep(60)
        # =========== GENERATE FINANCIAL HIGHLIGHTS
        finance_pairs_flat = list(zip(finance_pairs[1], finance_pairs[0]))  # [(r, q), (r, q), ...]
        section3 = self._sections(pairs=finance_pairs_flat)
        resp = self._answer(question=finance_formatting, ctx_text=section3)
        doc = insert_finance(resp['answer'], doc=doc)

        time.sleep(60)
        # =========== GENERATE CAPITAL STRUCTURE
        capital_pairs_flat = list(zip(capital_pairs[1], capital_pairs[0]))  # [(r, q), (r, q), ...]
        section4 = self._sections(pairs= capital_pairs_flat)
        resp = self._answer(question=capital_structure_formatting_2, ctx_text=section4)
        doc = insert_capital_structure(resp['answer'], doc=doc)

        pdf_bytes = docx_bytes_to_pdf_bytes(doc)

        return pdf_bytes
        # =========== UNION

from prompts4 import section7, finance_calculations, system_mod
import time
import re, time

company = 'SEAPORT_TOPCO_LIMITED'
sys = system_mod
calc = finance_calculations

agent = profileAgent(
    company_name = company,
    k=50, 
    max_text_recall_size=35, 
    max_chars=10000,
    model='gpt-5', 
    profile_prompt= sys,
    finance_calculations= calc
)


# =========== GENERATE BUSINESS OVERVIEW
stakeholders_pairs_flat = list(zip(stakeholders_pairs[1], stakeholders_pairs[0]))  # [(r, q), (r, q), ...]
section_built = agent._sections(pairs = stakeholders_pairs_flat)

#getting web search sections
new_section = f'All instructions applies to the company: {agent.company_name}\n\n{stakeholders_web} \n\n Mention in the Beggining of the answer that this is WEBSEARCH SOURCE'
messages = [
    {"role": "system", "content": default_gpt_prompt},
    {"role": "user",   "content": new_section},
]
resp_web = agent._web_search(messages)

section_built.append(resp_web)

# Join all context sections - they already contain their own citations
# Just concatenate them so the model can synthesize
ctx_text_formatted = "\n\n".join(section_built)

# resp = agent._answer(question=biz_overview_mix_formatting, ctx_text=ctx_text_formatted)
ctx_text_formatted

k is not a known attribute of class <class 'azure.search.documents._generated.models._models_py3.VectorizableTextQuery'> and will be ignored


KeyboardInterrupt: 

In [3]:
print(ctx_text_formatted)

Seaport Topco Limited is a private company.

- Immediate parent company: EQT Jupiter Luxco S.A.R.L (Luxembourg) [2024 Annual Report, Note 32 “Controlling party”, p.52; 2023 Annual Report, Note 30 “Controlling party”, p.46] [#1] [#3]
- Ultimate parent company: n.a. (not disclosed in the available filings). The filings note that “the smallest and largest Group in which the results of the Company are consolidated is that headed by Seaport Topco Limited,” but do not identify an ultimate controlling parent above EQT Jupiter Luxco S.A.R.L [2024 Annual Report, Note 32, p.52; 2023 Annual Report, Note 30, p.46] [#1] [#3]

Sources:
- Seaport Topco Limited Annual Report FY23 (approved Apr-24): Note 32 “Controlling party”, p.52 — https://aiprojec.../14171962/SEAPORT_TOPCO_LIMITED_AA_annualReport_2024-09-25_1.pdf [#1]
- Seaport Topco Limited Annual Report FY22 (published Oct-23): Note 30 “Controlling party”, p.46 — https://aiprojec.../14171962/SEAPORT_TOPCO_LIMITED_AA_annualReport_2023-10-15_2.pdf 

In [4]:
resp = agent._answer(question=stakeholders_web_mix, ctx_text=ctx_text_formatted)
print(resp)

{'answer': 'SECTION ONE – Key Stakeholders\n\nShareholders (private company)\n| Item | Name | Details |\n|---|---|---|\n| Immediate parent company | EQT Jupiter Luxco S.A.R.L. | Immediate parent; incorporated in Luxembourg. [#1, p.52]; [#3, p.46] |\n| Ultimate parent company | n.a. | Not disclosed in the available filings. The accounts state Seaport Topco Limited is the smallest and largest group in which the results are consolidated; no ultimate controlling party named above EQT Jupiter Luxco S.A.R.L. [#1, p.52]; [#3, p.46] |\n| Other registered shareholders (context) | Battery Ventures Select Fund II (AIV I Cayman), L.P.; Battery Investment Partners Select Fund II (AIV I Cayman), L.P.; various managers/employees (B/B1/C share classes) | Per latest Confirmation Statement (CS01, Jun-25), EQT Jupiter Luxco S.A.R.L. holds A Ordinary and A Preference shares; Battery funds hold minority A Ordinary and A Preference shares; management/employees hold B/B1/C classes. (Companies House CS01 link

In [5]:
print(resp['answer'])

SECTION ONE – Key Stakeholders

Shareholders (private company)
| Item | Name | Details |
|---|---|---|
| Immediate parent company | EQT Jupiter Luxco S.A.R.L. | Immediate parent; incorporated in Luxembourg. [#1, p.52]; [#3, p.46] |
| Ultimate parent company | n.a. | Not disclosed in the available filings. The accounts state Seaport Topco Limited is the smallest and largest group in which the results are consolidated; no ultimate controlling party named above EQT Jupiter Luxco S.A.R.L. [#1, p.52]; [#3, p.46] |
| Other registered shareholders (context) | Battery Ventures Select Fund II (AIV I Cayman), L.P.; Battery Investment Partners Select Fund II (AIV I Cayman), L.P.; various managers/employees (B/B1/C share classes) | Per latest Confirmation Statement (CS01, Jun-25), EQT Jupiter Luxco S.A.R.L. holds A Ordinary and A Preference shares; Battery funds hold minority A Ordinary and A Preference shares; management/employees hold B/B1/C classes. (Companies House CS01 link in Sources) |

Man

In [ ]:
import os, textwrap
import io

from typing import List, Dict, Optional
from xml.sax.saxutils import escape

from gpts.gpt_assistants import general_assistant
from dotenv import load_dotenv, find_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.core.exceptions import HttpResponseError
from azure.search.documents.models import HybridSearch


from openai import AzureOpenAI, APIConnectionError, OpenAI
from prompts import new_system_finance_prompt

from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer

from prompts4 import finance_calculations, finance_pairs, capital_pairs, stakeholders_pairs, biz_overview_pairs, revenue_pairs, default_gpt_prompt, section4a, section4b, section5, section3, biz_overview_web
from pages.design.func_tools import *
from pages.design.formatting import *
from pages.design.func_tools import docx_bytes_to_pdf_bytes
import re, time
 
load_dotenv(find_dotenv(), override=True)

# ---- Config (expects the same envs you already used) ----
SEARCH_ENDPOINT = os.environ["AZURE_SEARCH_ENDPOINT"]
SEARCH_INDEX    = os.environ["AZURE_SEARCH_INDEX"]
SEARCH_KEY      = os.getenv("AZURE_SEARCH_API_KEY")  # omit if using AAD/RBAC
VECTOR_FIELD    = os.getenv("VECTOR_FIELD")
TEXT_FIELD      = os.getenv("TEXT_FIELD")

AOAI_ENDPOINT   = os.environ["AZURE_OPENAI_ENDPOINT"]            # https://<resource>.openai.azure.com
AOAI_API_VER    = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-10-21")
AOAI_DEPLOYMENT = os.environ["AZURE_OPENAI_DEPLOYMENT"]          # e.g., gpt-4o-mini / o3-mini / gpt-5 preview
AOAI_KEY        = os.getenv("AZURE_OPENAI_API_KEY")              # omit if using AAD
OPENAI_API_KEY  = os.getenv("OPENAI_API_KEY")        # required

# ------------------ CODE

class profileAgent():

    """Hybrid (dense+sparse) RAG over Vector Store

    This Agent is responsible for creating Company Profiles. 
    It operates with gpt5.
    It is activated by a call on main rag when it is typed 'Create company profile'
    """

    def __init__(self, company_name, k, max_text_recall_size, max_chars, model, profile_prompt = new_system_finance_prompt, finance_calculations = finance_calculations):
        
        self.company_name = company_name

        self.k = k
        self.max_text_recall_size = max_text_recall_size
        self.model = model
        self.max_chars = max_chars

        self.azure_credentials = AzureKeyCredential(SEARCH_KEY) if SEARCH_KEY else DefaultAzureCredential()
        self.search_client = SearchClient(SEARCH_ENDPOINT, SEARCH_INDEX, credential=self.azure_credentials)

        self.az_openai = AzureOpenAI(azure_endpoint=AOAI_ENDPOINT, api_key=AOAI_KEY, api_version=AOAI_API_VER)
        self.profile_prompt = profile_prompt
        self.web_openai = OpenAI(api_key=OPENAI_API_KEY)

        self.reasoning_effort = "medium"
        self.verbosity = "medium"

        self.finance_calculations = finance_calculations

    def _company_filter(self) -> str:
        v = (self.company_name or "").replace("'", "''").strip()
        return f"company_name eq '{v}'" if v else None
    
    def assemble_bm25_from_llm(self, slots: dict) -> str:
        def q(s: str) -> str:
            # sanitize: remove internal quotes and trim
            s = (s or "").strip().replace('"', ' ')
            return f"\"{s}\"" if s else ""
        groups = []

        # must-have phrases (ANDed)
        for p in slots.get("must_have_phrases", []):
            qp = q(p)
            if qp:
                groups.append(qp)

        # metric / statement synonym groups (ORed within each group)
        for key in ["metric", "statement"]:
            syns = slots.get("synonyms", {}).get(key, []) or slots.get(key, [])
            syns = [q(s) for s in syns if s]
            if syns:
                groups.append("(" + " OR ".join(syns) + ")")

        return " AND ".join(groups) if groups else "\"financial statements\""


    def bm25_creator(self, prompt):

        instruction = (
            "Extract finance search slots for Azure AI Search. "
            "Return strict JSON: {\"metric\":[], \"statement\":[], \"synonyms\":{}, \"must_have_phrases\":[]} "
            "(include IFRS/US GAAP variants)."
        )
        resp = general_assistant(instruction, prompt, OPENAI_API_KEY, 'gpt-4o')

        try:
            slots = getattr(resp, "output_json", None)
            if slots is None:
                import json
                slots = json.loads(resp.output_text)
        except Exception:
            # fallback: minimal anchors from prompt
            slots = {"must_have_phrases": [prompt], "metric": [], "statement": [], "synonyms": {}}
        return self.assemble_bm25_from_llm(slots)

    def _retrieve_hybrid_enhanced(self, query_nl, k: int = 50, top_n = 30, fields=VECTOR_FIELD, max_text_recall_size:int = 800):
        sc = self.search_client
        flt = self._company_filter()
        
        try:
            vq = VectorizableTextQuery(text=query_nl, k=k, fields=VECTOR_FIELD)
            # Prefer vector-only search (integrated vectorization). If your index isn't set up for it, this raises.
            results = sc.search(
                search_text=self.bm25_creator(query_nl), 
                vector_queries=[vq], 
                top=top_n, 
                query_type="semantic",
                query_caption="extractive", 
                hybrid_search=HybridSearch(max_text_recall_size=self.max_text_recall_size),
                query_caption_highlight_enabled=True,
                filter=flt
                )
            mode = "hybrid + semantic"
        except HttpResponseError as e:
            # Fall back to lexical so you still get results while fixing vector config
            results = sc.search(search_text=self.bm25_creator(query_nl), top=k)
            mode = f"lexical (fallback due to: {e.__class__.__name__})"

        hits: List[Dict] = []
        for r in results:
            d = r.copy() if hasattr(r, "copy") else {k2: r[k2] for k2 in r}
            d["score"] = d.get("@search.reranker_score") or d.get("@search.score") or 0.0
            caps = d.get("@search.captions")
            if isinstance(caps, list) and caps:
                d["caption"] = getattr(caps[0], "text", None)
            hits.append(d)

        return mode, hits


    def _build_context(self, hits: List[Dict], text_field: str = TEXT_FIELD, max_chars: int = 20000):
        """Build a compact, numbered context block and also return the selected chunk metadata."""
        lines = []
        total = 0
        selected = []  # <- we'll return this

        for i, h in enumerate(hits, 1):
            title     = h.get("title")
            chunk_id  = h.get("chunk_id")
            full_text = (h.get(text_field) or "")
            if not full_text:
                continue

            preview = textwrap.shorten(full_text, width=700, placeholder=" ...")
            block = f"[{i}] title={title!r} | chunk_id={chunk_id} | score={h.get('score'):.4f}\n{full_text}"

            if total + len(block) > self.max_chars:
                break

            total += len(block)
            lines.append(block)

            # keep rich metadata so you can show or log it later
            selected.append({
                "i": i,
                "title": title,
                "chunk_id": chunk_id,
                "score": h.get("score"),
                "caption": h.get("caption"),
                "preview": preview,
                "text": full_text,  # full chunk text (not shortened)
                # include any other fields you index, if available:
                "metadata_storage_path": h.get("metadata_storage_path"),
                "page_number": h.get("page_number"),
                "doc_type": h.get("doc_type"),
            })

        return "\n\n---\n\n".join(lines), selected

        
    def _generate_pdf(self, text: str) -> bytes:

        buf = io.BytesIO()
        doc = SimpleDocTemplate(buf, pagesize=letter)
        styles = getSampleStyleSheet()
        body = styles["BodyText"]

        story = []
        # Treat double newlines as paragraph breaks; keep single newlines as <br/>
        for para in (text or "").split("\n\n"):
            safe = escape(para).replace("\n", "<br/>")
            story.append(Paragraph(safe if safe.strip() else "&nbsp;", body))
            story.append(Spacer(1, 8))

        doc.build(story)
        buf.seek(0)
        return buf.getvalue()
    
    def _extract_cited_idxs(self, answer: str) -> list[int]:
        # Matches [#1], [#12], etc. (also tolerates stray [1])
        nums = set(int(n) for n in re.findall(r"\[#?(\d+)\]", answer))
        return sorted(nums)

    def _rag_answer(self, rag_nl, question, k: int = 5, temperature: float = 0.2):

        # question = f'CREATE A SECTION OF COMPANY PROFILE USING LAST YEARS OF ANNUAL REPORT PRESENT IN THE CONTEXT FOR {self.company_name}. IF ANY INFORMATION IS NOT FOUND STATE AS n.a. .\n\n THIS IS THE SECTION TO BE BUILT: \n {section7}  \n USE THIS TO GUIDE YOURSELF ON SEMANTIC TERMS AND HOW TO CALCULATE: \n {finance_calculations}'
        
        mode, hits = self._retrieve_hybrid_enhanced(
            # query=rag_q, 
            query_nl=rag_nl,
            k=25
            )
        ctx_text, ctx_items = self._build_context(hits)

        system_msg = self.profile_prompt + (
            "\nWhen you use a fact from the context, add citations like [#1], [#2]."
            "\nOnly rely on the numbered context; if a value is missing, say 'n.a.'."
            f"\nIF ANY INFORMATION IS NOT FOUND STATE AS n.a. .\n\n USE THIS TO GUIDE YOURSELF ON SEMANTIC TERMS AND HOW TO CALCULATE: \n {finance_calculations}"
        )
        user_msg = f"Question:\n{question}\n\nContext snippets (numbered):\n{ctx_text}"

        client = self.az_openai
        messages = [
            {"role": "system", "content": system_msg},
            {"role": "user",   "content": user_msg},
        ]

        # Try streaming first (SSE). Some networks/proxies block streaming; if so, fall back.
        
        resp = client.chat.completions.create(
            model=AOAI_DEPLOYMENT,
            messages=messages,
            reasoning_effort="high"
        )
        answer = resp.choices[0].message.content
        mode_model = "non-streaming (fallback)"

        cited = self._extract_cited_idxs(answer)
        used_chunks = [c for c in ctx_items if c["i"] in cited]

        # return self._generate_pdf(answer)
        return {
            "answer": answer,
            "citations": cited,          # [1, 3, 7]
            "used_chunks": used_chunks,  # detailed dicts for each cited snippet
            "all_chunks": ctx_items,     # everything you sent (optional)
            "mode": mode                 # retrieval mode info (optional)
        }

    def _web_search(self, messages):
        resp = self.web_openai.responses.create(
            model='gpt-5',
            input=messages,
            tools=[{"type": "web_search"}],
            tool_choice="auto",
            # max_output_tokens=self.max_output_tokens,
            reasoning={"effort": self.reasoning_effort},
            text={"verbosity": self.verbosity},
        )
        
        return resp.output_text
    
    def _answer(self, question, ctx_text, k: int = 5, temperature: float = 0.2):

        system_msg = self.profile_prompt + (
            "\nWhen you use a fact from the context, preserve any existing citations like [#1], [#2], [#5, p.41] that are already in the context text."
            "\nOnly rely on the provided context; if a value is missing, say 'n.a.'."
            "\nIMPORTANT: If the formatting instructions request a Sources section, you MUST include it at the end."
            "\nFor the Sources section, list all citation numbers/references that appear in your answer, and describe what document/source each refers to based on information in the context."
        )
        user_msg = f"Question:\n{question}\n\nContext snippets:\n{ctx_text}"

        client = self.az_openai
        messages = [
            {"role": "system", "content": system_msg},
            {"role": "user",   "content": user_msg},
        ]

        # Try streaming first (SSE). Some networks/proxies block streaming; if so, fall back.

        resp = client.chat.completions.create(
            model=AOAI_DEPLOYMENT,
            messages=messages,
            reasoning_effort="high"
        )
        answer = resp.choices[0].message.content

        cited = self._extract_cited_idxs(answer)

        # return self._generate_pdf(answer)
        return {
            "answer": answer,
            "citations": cited,          # [1, 3, 7]
        }   
    
    @staticmethod
    def has_na(text: str) -> bool:
        # match "n.a." or "n/a" (case-insensitive)
        return bool(re.search(r"\b(n\.a\.|n/a)\b", text, flags=re.I))

    def _sections(self, pairs):

        answers = []

        max_extra_na_retries = 1        # try again at most 2 times (total <= 3 calls per item)
        base_delay_seconds = 3.0        # polite delay between attempts


        for q, r in pairs:
            tries = 0
            while True:
                if tries > 0:
                    # small incremental delay before re-trying
                    time.sleep(base_delay_seconds + 0.5 * tries)

                resp = self._rag_answer(rag_nl=r[0], question=q[0])
                answer_text = resp["answer"]

                # stop if good answer OR we've exhausted retries
                if not profileAgent.has_na(answer_text) or tries >= max_extra_na_retries:
                    answers.append(answer_text)
                    break

                # otherwise, try again
                tries += 1

            # optional small gap between different (r,q) items
            time.sleep(5.0)
        
        return answers
    
    def _generate_section(self, section):

        if section == 'GENERATE BUSINESS OVERVIEW':
            # =========== GENERATE BUSINESS OVERVIEW
            biz_overview_pairs_flat = list(zip(biz_overview_pairs[1], biz_overview_pairs[0]))  # [(r, q), (r, q), ...]
            section_built = self._sections(pairs = biz_overview_pairs_flat)

            #getting web search sections
            new_section = f'All instructions applies to the company: {self.company_name}\n\n{biz_overview_web} \n\n Mention in the Beggining of the answer that this is WEBSEARCH SOURCE'
            messages = [
                {"role": "system", "content": default_gpt_prompt},
                {"role": "user",   "content": new_section},
            ]
            resp_web = self._web_search(messages)

            section_built.append(resp_web)

            # Join all context sections - they already contain their own citations
            # Just concatenate them so the model can synthesize
            ctx_text_formatted = "\n\n".join(section_built)

            resp = self._answer(question=biz_overview_mix_formatting, ctx_text=ctx_text_formatted)
            return resp['answer']
        elif section == 'GENERATE KEY STAKEHOLDERS':
        # =========== GENERATE KEY STAKEHOLDERS
            stakeholders_pairs_flat = list(zip(stakeholders_pairs[1], stakeholders_pairs[0]))  # [(r, q), (r, q), ...]
            section_built = self._sections(pairs= stakeholders_pairs_flat)
            resp = self._answer(question=stakeholders_formatting_2, ctx_text=section_built)
            return resp['answer']
        elif section == 'GENERATE FINANCIAL HIGHLIGHTS':
            # =========== GENERATE FINANCIAL HIGHLIGHTS
            finance_pairs_flat = list(zip(finance_pairs[1], finance_pairs[0]))  # [(r, q), (r, q), ...]
            section_built = self._sections(pairs=finance_pairs_flat)
            resp = self._answer(question=finance_formatting_2, ctx_text=section_built)
            return resp['answer']
        elif section == 'GENERATE CAPITAL STRUCTURE':
            # =========== GENERATE CAPITAL STRUCTURE
            capital_pairs_flat = list(zip(capital_pairs[1], capital_pairs[0]))  # [(r, q), (r, q), ...]
            section_built = self._sections(pairs= capital_pairs_flat)
            resp = self._answer(question=capital_structure_formatting_2, ctx_text=section_built)
            return resp['answer']
        elif section == 'GENERATE REVENUE SPLIT':
            # =========== GENERATE CAPITAL STRUCTURE
            revenue_pairs_flat = list(zip(revenue_pairs[1], revenue_pairs[0]))  # [(r, q), (r, q), ...]
            section_built = self._sections(pairs= revenue_pairs_flat)
            resp = self._answer(question=section3, ctx_text=section_built)
            return resp['answer']
        elif section == 'GENERATE PRODUCTS SERVICES OVERVIEW':
            # =========== GENERATE CAPITAL STRUCTURE
            new_section = f'All instructions applies to the company: {self.company_name}\n\n{section4a}'
            messages = [
                {"role": "system", "content": default_gpt_prompt},
                {"role": "user",   "content": new_section},
            ]
            resp = self._web_search(messages)
            return resp 
        elif section == 'GENERATE GEO FOOTPRINT':
            # =========== GENERATE CAPITAL STRUCTURE
            new_section = f'All instructions applies to the company: {self.company_name}\n\n{section4b}'
            messages = [
                {"role": "system", "content": default_gpt_prompt},
                {"role": "user",   "content": new_section},
            ]
            resp = self._web_search(messages)
            return resp
        elif section == 'GENERATE DEVELOPMENTS HIGHLIGHTS':
            # =========== GENERATE CAPITAL STRUCTURE
            new_section = f'All instructions applies to the company: {self.company_name}\n\n{section5}'
            messages = [
                {"role": "system", "content": default_gpt_prompt},
                {"role": "user",   "content": new_section},
            ]
            resp = self._web_search(messages)
            return resp


    def generate_company_profile(self):

        # =========== GENERATE BUSINESS OVERVIEW
        biz_overview_pairs_flat = list(zip(biz_overview_pairs[1], biz_overview_pairs[0]))  # [(r, q), (r, q), ...]
        section1 = self._sections(pairs = biz_overview_pairs_flat)
        resp = self._answer(question=business_overview_formatting, ctx_text=section1)
        doc = insert_biz_overview(resp['answer'])

        time.sleep(60)
        # =========== GENERATE KEY STAKEHOLDERS
        stakeholders_pairs_flat = list(zip(stakeholders_pairs[1], stakeholders_pairs[0]))  # [(r, q), (r, q), ...]
        section2 = self._sections(pairs= stakeholders_pairs_flat)
        resp = self._answer(question=stakeholders_formatting, ctx_text=section2)
        doc = insert_stakeholders(resp['answer'], doc=doc)
        
        time.sleep(60)
        # =========== GENERATE FINANCIAL HIGHLIGHTS
        finance_pairs_flat = list(zip(finance_pairs[1], finance_pairs[0]))  # [(r, q), (r, q), ...]
        section3 = self._sections(pairs=finance_pairs_flat)
        resp = self._answer(question=finance_formatting, ctx_text=section3)
        doc = insert_finance(resp['answer'], doc=doc)

        time.sleep(60)
        # =========== GENERATE CAPITAL STRUCTURE
        capital_pairs_flat = list(zip(capital_pairs[1], capital_pairs[0]))  # [(r, q), (r, q), ...]
        section4 = self._sections(pairs= capital_pairs_flat)
        resp = self._answer(question=capital_structure_formatting_2, ctx_text=section4)
        doc = insert_capital_structure(resp['answer'], doc=doc)

        pdf_bytes = docx_bytes_to_pdf_bytes(doc)

        return pdf_bytes
        # =========== UNION


In [8]:

import io, tempfile, subprocess, shutil
from pathlib import Path
from typing import Optional, Tuple, Union

from docx import Document
from docx.enum.text import WD_BREAK


biz_output = """
Here are the requested variables for Seaport Topco Limited, with sources and page references:

- Primary Activity:
  - “The principal activity of the Group continues to include the research and development of pharmaceutical instrumentation.” Source: Annual Report (FY23), Strategic/Directors’ Report, p.5 [1][2].

- Business Review:
  - FY24 Group Strategic Report (Review of the business and future developments):
    - Cash and cash equivalents decreased from £11.8m at Dec-23 to £11.8m at Dec-24; cash generated from operating activities was £10.4m (FY23: £15.3m).
    - Stock reduced to £13.2m at Dec-24 (Dec-23: £15.4m).
    - Trade debtors increased to £17.0m at Dec-24 (Dec-23: £13.2m) due to higher late-year revenue.
    - Creditors due within one year increased to £64.7m at Dec-24 (Dec-23: £46.6m), driven by new loans (£7.7m) and a net RCF draw (£9.2m).
    - Deferred tax liability decreased, largely due to decreases on intangible fixed assets (£4.7m) and a charge related to losses on consolidation (£4.8m).
    - Headcount rose to 418 (from 405).
    - Source: Annual Report (FY24), Group Strategic Report, “Review of the business and future developments (continued)”, within Group Strategic Report pages 1–5; exact page n.a. [9]. For section pagination reference, see FY23 contents page indicating Group Strategic Report spans pp.1–5 [3][4].

- Introduction:
  - n.a. (No explicit “Introduction” section text available in the provided excerpts) [3][4][9].

- Bank Debt/Borrowings/Creditors:
  - Creditors: amounts falling due within one year: £46.6m at Dec-23 (restated Dec-22: £35.4m) — Note 21, Consolidated Statement of Financial Position, p.14 [7][8].
  - Creditors: amounts falling due after more than one year: £168.7m at Dec-23 (restated Dec-22: £171.7m) — Note 22, Consolidated Statement of Financial Position, p.14 [7][8].
  - FY24 movement: Creditors due within one year increased to £64.7m at Dec-24, driven by £7.7m of new loans and a £9.2m net draw on the revolving credit facility — Group Strategic Report (FY24), “Review of the business and future developments (continued)”, exact page n.a. (within pp.1–5) [9].
"""

cap_output = r"""
Metric,FY24,FY23,FY22
"Facility Name","n.a.","Facility B1 (EUR term loan); Facility B2 (USD term loan); Revolving Credit Facility; Delayed Drawdown Facility [#2][#3]","Revolving Credit Facility; Delayed Drawdown Facility [#3]"
"Interest Rate","n.a.","Euribor + 6.25%; Term SOFR + 6.25% [#2][#3]","n.a."
Maturity,"n.a.","Aug-29 [#2][#3]","n.a."
"Adjusted EBITDA","n.a.","£17.9m [#2][#3]","£10.6m [#2][#3]"
"Cash (Closing Cash)","n.a.","£11.8m [#2][#3]","n.a."
"Net Debt","n.a.","£171.9m [#2][#3]","n.a."
Liquidity,"n.a.","£25.8m [#2][#3]","n.a."
"Leverage (Net Debt/EBITDA)","n.a.","9.6x [#2][#3]","n.a."
"Facility B1 outstanding (GBP)","n.a.","£36.0m [#2][#3]","n.a."
"Facility B2 outstanding (GBP)","n.a.","£135.0m [#2][#3]","n.a."
"RCF drawn","n.a.","£16.0m [#2][#3]","£16.0m [#3]"
"RCF facility size","n.a.","£30.0m [#2][#3]","£30.0m [#3]"
"Delayed Drawdown Facility size","n.a.","£75.0m [#2][#3]","£75.0m [#3]"
"Bank loans due after >5 years","n.a.","£168.7m [#2][#3]","n.a."
"Bank loans due within 1 year","n.a.","£14.7m [#2][#3]","n.a."
"Bank loans + RCF outstanding (excl. leases)","n.a.","£187.0m [#2][#3]","n.a."

Summary / Interpretation
- FY23 leverage is high at 9.6x, based on £171.9m net debt and £17.9m Adjusted EBITDA.
- FY23 facility mix is dominated by term loans (Facility B1 ~£36.0m and B2 ~£135.0m) maturing in Aug-29, plus a £30.0m RCF (of which £16.0m was drawn) and a £75.0m delayed draw facility.
- FY23 liquidity appears modest at £25.8m, combining £11.8m closing cash with remaining headroom on the £30.0m RCF (drawn £16.0m).
- Maturity profile in FY23 is back-ended: £168.7m due after >5 years versus £14.7m due within 1 year; both term facilities mature in Aug-29.
- Total FY23 bank loans + RCF outstanding (excl. leases) sums to £187.0m, indicating a sizeable secured debt stack.
- FY24 disclosures are not available in the provided excerpts, and FY22 data is limited beyond RCF details.

Sources
- [#2] Seaport Topco Limited Annual Report (file date 25-Sep-24), pp. 8, 45, 52 — p.8 Adjusted EBITDA (£17.9m FY23; £10.6m FY22); p.45 Loans note (Facility B1/B2 amounts, Aug-29 maturities, interest margins; RCF £30.0m and ~£16.0m drawn; £75.0m delayed draw facility; bank loans due >5 years £168.7m and within 1 year £14.7m); p.52 Net debt analysis (Net Debt £171.9m; closing cash £11.8m). Link: https://aiprojectteneo.blob.core.windows.net/companieshouselinglefile/14171962/SEAPORT_TOPCO_LIMITED_AA_annualReport_2024-09-25_0.pdf
- [#3] Seaport Topco Limited Annual Report (file date 25-Sep-24), pp. 8, 36, 45, 52 — corroborates Adjusted EBITDA figures; Going concern (p.36) notes RCF drawn £16.0m at Dec-22; Loans note (p.45) for facility sizes/draws and maturities; Net debt analysis (p.52) for Net Debt and closing cash. Link: https://aiprojectteneo.blob.core.windows.net/companieshousinglefile/14171962/SEAPORT_TOPCO_LIMITED_AA_annualReport_2024-09-25_1.pdf
"""

stakeholders_output = r"""
Metric,Shareholders
"Shareholders",n.a.
"Management","Directors (FY24): M Bauer; R Diggelmann — resigned Dec-24; P Dowdy — resigned Feb-25; J Feldman; R Friel — resigned Mar-25; K Murphy; D Newble — resigned Jul-24; A Thorburn; R Walton — appointed Jul-24"
"Lenders",n.a.
"Auditors","Grant Thornton UK LLP (Statutory Auditor); Stephen Wyborn (Senior Statutory Auditor)"
"Advisors","Facility agent: Kroll Agency Services Limited; Bankers: n.a.; Solicitors: n.a.; Financial advisor: n.a."

Summary / Interpretation
- Shareholder information is n.a., indicating no disclosed immediate or ultimate parent in the provided filings.
- Management is represented by the FY24 directors list; specific Chairman/CEO/CFO titles are not provided.
- Lenders are n.a., suggesting no disclosed bank facilities/borrowings in the available excerpts.
- Auditors are identified (Grant Thornton UK LLP; Senior Statutory Auditor Stephen Wyborn), while most other advisors are n.a. except the facility agent (Kroll Agency Services Limited).
- Advisor disclosure is limited (bankers/solicitors/financial advisor n.a.), constraining visibility into counterparties.

SECTION 3 - SOURCES
- [#1] Seaport Topco Limited AA Annual Report (published Sep-25), Directors’ Report for the year ended 31 Dec-24, p.12. Link: https://aiprojectteneo.blob.core.windows.net/companieshousesinglefile/14171962/SEAPORT_TOPCO_LIMITED_AA_annualReport_2025-09-30_0.pdf — used for the “Management” (directors) row.
- [#2] Seaport Topco Limited AA Annual Report (2024-09-25) – provided excerpt indicating Company Information page not available — supports n.a. for certain advisor details.
- [#3] Seaport Topco Limited annual report (FY24), Notes to the Financial Statements – Accounting policies excerpt (page n.a.) — used to check terminology (bank loans/borrowings); lenders n.a. and facility agent reference noted elsewhere.
- [#4] Seaport Topco Limited Annual Report 2024, Independent Auditors’ Report signature block, file: SEAPORT_TOPCO_LIMITED_AA_annualReport_2024-09-25_1.pdf (26 pages), signed Apr-24 — used for “Auditors” identification and to support shareholder n.a.
- [#5] Seaport Topco Limited Annual Report (published Sep-24), Notes to the Financial Statements, Note 16: Fixed asset investments – Indirect subsidiary undertakings (page n.a.) — used to support “Shareholders” n.a. (no parent/ultimate controlling party disclosed).
- [#7] Seaport Topco Limited Annual Report 2024, Independent Auditors’ Report signature block, file: SEAPORT_TOPCO_LIMITED_AA_annualReport_2024-09-25_0.pdf (26 pages), signed Apr-24 — corroborates “Auditors” identification.
"""

finance_output = """
Metric,FY24,FY23,FY22
"Revenue (Turnover)","£576.8m [#6]","£81.435m [#6]","n.a."
"Revenue growth % (yoy)","+608.6% [#6]","n.a.","n.a."
"Cost of sales","n.a.","£33.0m [#3]","£18.5m [#3]"
"Gross profit","£43.9m [#6]","£48.4m [#3]","£14.3m [#3]"
"Gross margin %","7.6% [#6]","59.5% [#3][#6]","n.a."
"EBITDA","n.a.","£20.2m [#4]","£(1.2)m [#4]"
"EBITDA margin %","n.a.","24.8% [#4][#6]","n.m. [#4]"
"Operating profit","n.a.","£(273.1)m [#4]","£(22.9)m [#4]"
"Depreciation","n.a.","£2.0m [#4]","£0.6m [#4]"
"Amortization","n.a.","£60.6m [#4]","£21.2m [#4]"
"Net cash flow from operating activities","£9.6m [#11]","£15.3m [#1]","£(1.8)m [#1]"
"Net working capital (cash flow changes)","n.a.","£3.9m [#7]","£(3.0)m [#7]"
"Cash flow from operating activities excl. working capital","n.a.","£11.4m [#1][#7]","£1.2m [#1][#7]"
"Net cash flow from investing activities","£(2.7)m [#11]","£(8.0)m [#1]","£(639.0)m [#1]"
"Capex","n.a.","£7.2m [#9]","£0.9m [#9]"
"Other cash flow from investing activities","n.a.","£(15.2)m [#1][#9]","£(639.9)m [#1][#9]"
"CFADS","£6.9m [#11]","£7.3m [#1]","£(640.8)m [#1]"
"Net cash flow from financing activities","n.a.","£(4.2)m [#10]","£657.6m [#10]"
"Debt issuance (draw down of bank loans)","n.a.","£12.1m [#10]","£187.3m [#10]"
"Share issuance","n.a.","£0.4m [#10]","£484.2m [#10]"
"Opening cash","£11.8m [#10]","n.a.","n.a."
"Change in cash","n.a.","n.a.","n.a."
"Closing cash","n.a.","£11.8m [#12]","£9.0m [#12]"
"Total debt","n.a.","£183.4m [#13]","n.a."
"Net debt","n.a.","£171.6m [#13]","n.a."
"Leverage (Net Debt/EBITDA)","n.a.","8.5x [#13][#4]","n.m. [#4]"

Summary / Interpretation
- FY24 shows a dramatic revenue surge (+608.6% yoy) versus FY23, while gross margin compresses sharply to 7.6% from 59.5%, indicating significant mix/pricing shifts.
- Operating performance improved from FY22 to FY23 (EBITDA turning from negative to £20.2m and a 24.8% margin), but FY24 EBITDA is n.a., limiting margin trend analysis for the latest year.
- CFADS is positive in both FY24 (£6.9m) and FY23 (£7.3m), contrasting with the large negative CFADS in FY22 (£(640.8)m) driven by heavy investing outflows.
- FY23 working-capital movements (£3.9m outflow) reduced reported operating cash; excluding working capital, FY23 operating cash was £11.4m versus £1.2m in FY22.
- FY23 leverage is high at 8.5x (Net debt £171.6m vs EBITDA £20.2m), highlighting balance sheet pressure despite improved operating earnings.
- Several FY24 balance-sheet/cash items (EBITDA, debt, closing cash) are n.a., so liquidity/leverage assessments for FY24 rely mainly on cash flow and top-line signals.

SECTION 3 - SOURCES
- [#1] Seaport Topco Limited 2024 Annual Report (filed Sep-24), Consolidated Statement of Cash Flows, p.18 — FY23 net cash from operating activities (£15.3m) and net cash used in investing (£(8.0)m); FY22 net cash used in operating (£(1.8)m) and investing (£(639.0)m). Source reference within context: p.18 [4].
- [#3] Seaport Topco Limited 2024 Annual Report (Sep-24), Consolidated Statement of Comprehensive Income, p.13 — FY23/FY22 cost of sales (£33.0m/£18.5m) and gross profit (£48.4m/£14.3m). Link: https://approjectteneo.blob.core.windows.net/companyhousesinglefile/14171962/SEAPORT_TOPCO_LIMITED_AA_annualReport_2024-09-25_0.pdf and duplicate [#4].
- [#4] Seaport Topco Limited AA annual report (2024-09-25), p.9 — FY23/FY22 operating loss (‑£273.1m/‑£22.9m), depreciation (£2.0m/£0.6m), amortization (£60.6m/£21.2m), and EBITDA (£20.2m/£(1.2)m).
- [#6] Seaport Topco Limited Consolidated Statement of Comprehensive Income for the year ended 31 Dec-24 (SEAPORT_TOPCO_LIMITED_AA_annualReport_2025-09-30_0.pdf), p.12 — FY24 revenue (£576.8m), gross profit (£43.9m), and gross margin (7.6%); FY23 revenue referenced at £81.435m.
- [#7] Seaport Topco Limited Annual Report (Sep-24), Consolidated Statement of Cash Flows, p.18 — Working-capital movement lines (stocks, debtors, creditors) used to compute NWC for FY23 (£3.9m) and FY22 (£(3.0)m).
- [#9] Seaport Topco Limited 2024 Annual Report, Consolidated Statement of Cash Flows, p.18 — Capex components: purchase of tangible and intangible fixed assets (FY23: £6.2m and £1.0m; FY22: £0.9m and n.a.). Links: https://aprojectteneo.blob.core.windows.net/companieshousesinglefile/14171962/SEAPORT_TOPCO_LIMITED_AA_annualReport_2024-09-25_0.pdf and duplicate [#5].
- [#10] Seaport Topco Limited 2024 Annual Report, Consolidated Statement of Cash Flows (continued), p.19 — FY23 net cash from financing (£(4.2)m), debt issuance (£12.1m), share issuance (£0.4m); FY22 net financing (£657.6m), debt issuance (£187.3m), share issuance (£484.2m); FY24 opening cash (£11.8m).
- [#12] Seaport Topco Limited 2024 Annual Report, Consolidated Statement of Cash Flows, p.19 — Closing cash: FY23 £11.8m and FY22 £9.0m.
- [#13] Seaport Topco Limited 2024 Annual Report, Note 31 “Analysis of net debt,” p.52 — FY23 total debt (£183.4m) and net debt (£171.6m).
- [#11] Seaport Topco Limited Annual Report 2025 (partial excerpt), Consolidated Statement of Cash Flows, p.17 — FY24 net cash from operating activities (£9.6m) and net cash used in investing (£(2.7)m); “Change in cash” not shown.
"""

def _docx_bytes_to_pdf_bytes_with_docx2pdf(docx_bytes: bytes) -> Optional[bytes]:
    try:
        from docx2pdf import convert  # requires MS Word (Windows/macOS)
    except Exception:
        return None
    try:
        with tempfile.TemporaryDirectory() as td:
            in_path  = Path(td) / "doc.docx"
            out_path = Path(td) / "doc.pdf"
            in_path.write_bytes(docx_bytes)
            convert(str(in_path), str(out_path))  # Word/Automator
            return out_path.read_bytes() if out_path.exists() else None
    except Exception:
        return None

def _docx_bytes_to_pdf_bytes_with_lo(docx_bytes: bytes) -> Optional[bytes]:
    soffice = shutil.which("soffice") or shutil.which("libreoffice")
    if not soffice:
        return None
    try:
        with tempfile.TemporaryDirectory() as td:
            in_path  = Path(td) / "doc.docx"
            out_dir  = Path(td) / "out"
            out_dir.mkdir(parents=True, exist_ok=True)
            in_path.write_bytes(docx_bytes)
            r = subprocess.run(
                [soffice, "--headless", "--convert-to", "pdf", "--outdir", str(out_dir), str(in_path)],
                check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE
            )
            out_path = out_dir / "doc.pdf"
            return out_path.read_bytes() if out_path.exists() else None
    except Exception:
        return None

def docx_bytes_to_pdf_bytes(docx_bytes: bytes) -> Optional[bytes]:
    return (_docx_bytes_to_pdf_bytes_with_docx2pdf(docx_bytes)
            or _docx_bytes_to_pdf_bytes_with_lo(docx_bytes))

def document_to_docx_bytes(doc: Document) -> bytes:
    buf = io.BytesIO()
    doc.save(buf)
    return buf.getvalue()

doc = insert_biz_overview(gpt_output=biz_output)

doc = insert_stakeholders(stakeholders_output, doc=doc)

doc = insert_finance(finance_output, doc=doc)

# doc = insert_capital_structure(cap_output, doc=doc)

# doc2 = docx_bytes_to_pdf_bytes(doc)

# docx_bytes = document_to_docx_bytes(doc)

# pdf_bytes = docx_bytes_to_pdf_bytes(docx_bytes)


doc.save("report.docx")

def save_docx_to_pdf_via_libreoffice(doc, pdf_path: str):
    tmpdir = tempfile.mkdtemp()
    docx_path = os.path.join(tmpdir, "report.docx")
    doc.save(docx_path)

    outdir = str(Path(pdf_path).parent)
    os.makedirs(outdir, exist_ok=True)

    # Convert using LibreOffice in headless mode
    cmd = [
        "soffice", "--headless", "--nologo",
        "--convert-to", "pdf:writer_pdf_Export",
        "--outdir", outdir, docx_path
    ]
    res = subprocess.run(cmd, capture_output=True, text=True, timeout=120)

    produced = os.path.join(outdir, "report.pdf")
    if res.returncode != 0 or not os.path.exists(produced):
        raise RuntimeError(f"LibreOffice failed:\nSTDOUT:\n{res.stdout}\nSTDERR:\n{res.stderr}")

    shutil.move(produced, pdf_path)
    shutil.rmtree(tmpdir, ignore_errors=True)

save_docx_to_pdf_via_libreoffice(doc, "report.pdf")


Updated document written
Updated document written
Updated document written
WARNING — CSV metrics not matched to any row:
 - Revenue (Turnover)
 - Revenue growth % (yoy)
 - Cost of sales
 - Operating profit
 - Depreciation
 - Amortization
 - Net cash flow from operating activities
 - Net working capital (cash flow changes)
 - Cash flow from operating activities excl. working capital
 - Net cash flow from investing activities
 - Net cash flow from financing activities
 - Debt issuance (draw down of bank loans)
 - Share issuance
 - Leverage (Net Debt/EBITDA)


FileNotFoundError: [Errno 2] No such file or directory: 'soffice'

In [89]:
from prompts4 import section7, finance_calculations, system_mod
import time
import re, time

company = 'SEAPORT_TOPCO_LIMITED'
sys = system_mod
calc = finance_calculations

agent = profileAgent(
    company_name = company,
    k=50, 
    max_text_recall_size=35, 
    max_chars=10000,
    model='gpt-5', 
    profile_prompt= sys, 
    finance_calculations= calc
)

pdf = agent.generate_company_profile()

k is not a known attribute of class <class 'azure.search.documents._generated.models._models_py3.VectorizableTextQuery'> and will be ignored


KeyboardInterrupt: 

In [1]:
pip install langgraph


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [92]:
from prompts4 import section7, finance_calculations, system_mod
import time
import re, time


company = 'SEAPORT_TOPCO_LIMITED'
sys = system_mod
calc = finance_calculations

agent = profileAgent(
    company_name = company,
    k=25, 
    max_text_recall_size=35, 
    max_chars=10000,
    model='gpt-5', 
    profile_prompt= sys, 
    finance_calculations= calc
)

# rag_q = "FIND THE VARIABLES 'Net cash from operating activities' and 'Net cash used in investing activities' in the statement of cash flows. FILES FROM 2024."
# q = 'CFADS (calc. Net cash from operating activities + Net cash used in investing activities). Show me the formula with values and final result.'


def has_na(text: str) -> bool:
    # match "n.a." or "n/a" (case-insensitive)
    return bool(re.search(r"\b(n\.a\.|n/a)\b", text, flags=re.I))

answers = []

max_extra_na_retries = 1        # try again at most 2 times (total <= 3 calls per item)
base_delay_seconds = 3.0        # polite delay between attempts

biz_overview_pairs_flat = list(zip(biz_overview_pairs[1], biz_overview_pairs[0]))  # [(r, q), (r, q), ...]
stakeholders_pairs_flat = list(zip(stakeholders_pairs[1], stakeholders_pairs[0]))  # [(r, q), (r, q), ...]
capital_pairs_flat = list(zip(capital_pairs[1], capital_pairs[0]))  # [(r, q), (r, q), ...]
finance_pairs_flat = list(zip(finance_pairs[1], finance_pairs[0]))  # [(r, q), (r, q), ...]capital_pairs

# finance_pairs_flat = list(zip(finance_pairs[1], finance_pairs[0]))  # [(r, q), (r, q), ...]
        
for q, r in finance_pairs_flat:
    tries = 0
    while True:
        if tries > 0:
            # small incremental delay before re-trying
            time.sleep(base_delay_seconds + 0.5 * tries)
        
        print(r)
        print(q)
        resp = agent._rag_answer(rag_nl=r[0], question=q[0])
        answer_text = resp["answer"]
        # answer_text = 'oi'

        # stop if good answer OR we've exhausted retries
        # if not has_na(answer_text) or tries >= max_extra_na_retries:
        #     answers.append(answer_text)
        #     break
        if answer_text:
            answers.append(answer_text)
            break

        # otherwise, try again
        tries += 1

    # optional small gap between different (r,q) items
    time.sleep(5.0)

# 11m 38

k is not a known attribute of class <class 'azure.search.documents._generated.models._models_py3.VectorizableTextQuery'> and will be ignored


["FIND THE VARIABLES 'Net cash from operating activities' and 'Net cash used in investing activities' in the statement of cash flows. FILES FROM 2024."]
['CFADS (calc. Net cash from operating activities + Net cash used in investing activities). Show me the formula with values and final result.']


k is not a known attribute of class <class 'azure.search.documents._generated.models._models_py3.VectorizableTextQuery'> and will be ignored


["FIND THE VARIABLES 'Revenue'/'Turnover'/'Turn Over' in the Income statement. FILES FROM 2024."]
['Revenue/Turnover/Turn over (Use Income Statement – Always Given)']


HttpResponseError: () Could not complete vectorization action. The vectorization endpoint returned status code '429' (TooManyRequests).
Code: 
Message: Could not complete vectorization action. The vectorization endpoint returned status code '429' (TooManyRequests).

In [ ]:
def pdf_processing(prompt):#####new
    from io import BytesIO
    from typing import Tuple, Annotated, TypedDict
    import time
    import streamlit as st
    import sys, pathlib
 
    # Ensure repo root is importable
    repo_root = pathlib.Path(__file__).resolve().parent.parent
    if str(repo_root) not in sys.path:
        sys.path.insert(0, str(repo_root))
 
    # Engine
    from engines.engine import HybridEngine
 
    # LangGraph
    from langgraph.graph import StateGraph, START, END
    from langgraph.graph.message import add_messages
    from langgraph.prebuilt import ToolNode, tools_condition
    from langgraph.checkpoint.memory import MemorySaver
 
    # LangChain
    from langchain_core.tools import tool
    from langchain_openai import ChatOpenAI
 
    # Cache builder
    # @st.cache_resource(show_spinner=False)  # disable while debugging
    def ocr_engine_cached_multi(files_bytes: Tuple[bytes, ...], files_names: Tuple[str, ...]):
        pdf_streams = tuple((BytesIO(b), n) for b, n in zip(files_bytes, files_names))
        engine = HybridEngine(pdf_streams)
        t0 = time.perf_counter(); engine.main(); build_s = time.perf_counter() - t0
        timings = getattr(engine, "timings", {})
        timings["total_build_s"] = build_s
        return engine, timings
 
    # LangGraph builder
    memory = MemorySaver()
    class State(TypedDict):
        messages: Annotated[list, add_messages]
 
    def build_graph(engine: HybridEngine):
        @tool
        def pdf_search(query: str) -> str:
            """Retrieve top snippets from the indexed PDFs for a query."""
            docs = engine.hybrid.get_relevant_documents(query)
            if not docs:
                return "NO_MATCH"
            return "\n---\n".join([d.page_content[:500] for d in docs[:3]])
       
        @tool
        def web_search(query: str) -> str:
            """Use OpenAI Responses' built-in web_search to fetch up to 3 results.
            Format:
            [web] <title> — <one-line snippet> <url>
            If nothing found or on error, return WEB_NO_RESULTS.
            """
            print(f"[DEBUG] web_search called with query: {query}")
            try:
                client = OpenAI()
                resp = client.responses.create(
                    model="gpt-5",
                    tools=[{"type": "web_search"}],
                    input=(
                        f"Search the web for: {query}\n"
                        "Return up to 3 bullets, each exactly as:\n"
                        "[web] <title> — <one-line snippet> <url>\n"
                        "If nothing is found, return exactly: WEB_NO_RESULTS"
                    ),
                )
                text = (getattr(resp, "output_text", "") or "").strip()
                print("[DEBUG] responses output_text len:", len(text))
                if not text:
                    return "WEB_NO_RESULTS"
                bullets = [ln.strip() for ln in text.split("\n") if ln.strip().startswith("[web] ")][:3]
                if not bullets:
                    return "WEB_NO_RESULTS"
                print(f"[DEBUG] web_search returning {len(bullets)} results")
                return "\n---\n".join(bullets)
            except Exception as e:
                print(f"[ERROR] web_search failed: {e}")
                return "WEB_NO_RESULTS"
 
        # If you already built a real calc tool elsewhere, import and wrap it here.
        # Example: engines/calc_tool.py defines def calc_script(data: str) -> str: ...
        try:
            from engines.calc_tool import calc_script as _calc_impl  # <-- adjust path if different
            @tool
            def calc_script(data: str) -> str:
                """Financial calculations tool."""
                return _calc_impl(data)
        except Exception:
            @tool
            def calc_script(data: str) -> str:
                """Financial calculations tool (placeholder if module not found)."""
                print(f"[DEBUG] calc_script called with data: {data}")
                return "CALC_NOT_IMPLEMENTED"
 
        tools = [pdf_search, web_search, calc_script]
        llm = ChatOpenAI(model="gpt-5")
        llm_with_tools = llm.bind_tools(tools)
 
        def chatbot(state: State) -> State:
            ai_msg = llm_with_tools.invoke(state["messages"])
            return {"messages": [ai_msg]}
 
        builder = StateGraph(State)
        builder.add_node("chatbot", chatbot)
        builder.add_node("tools", ToolNode(tools))
        builder.add_edge(START, "chatbot")
        builder.add_conditional_edges("chatbot", tools_condition)
        builder.add_edge("tools", "chatbot")
        builder.add_edge("chatbot", END)
        return builder.compile(checkpointer=memory)
 
    def main():
        st.set_page_config(page_title="Oraculum")
        st.title("Oraculum")
        st.write("✅ App booted")  # prove we rendered
 
        # Safe init
        defaults = {
            "ocr_mode": False,
            "text_engine": False,
            "processed": False,
            "ocr_engine": None,
            "graph": None,
            "ocr_timings": {},
        }
        for k, v in defaults.items():
            if k not in st.session_state:
                st.session_state[k] = v
 
        pdf_files = st.file_uploader("Upload your PDF(s)", type=["pdf"], accept_multiple_files=True)
        col1, col2 = st.columns(2)
        if col1.button("OCR Engine"):
            st.session_state.update({"ocr_mode": True, "text_engine": False, "processed": False})
 
        # --- OCR Engine (multi-file) ---
        if st.session_state.get("ocr_mode", False) and pdf_files:
            files_bytes: Tuple[bytes, ...] = tuple(f.getvalue() for f in pdf_files)
            files_names: Tuple[str, ...] = tuple(f.name for f in pdf_files)
 
            if not st.session_state.get("processed", False):
                with st.spinner("Building OCR (hybrid) index across all PDFs..."):
                    try:
                        engine, timings = ocr_engine_cached_multi(files_bytes, files_names)
                        st.session_state.ocr_engine = engine
                        st.session_state.ocr_timings = timings
                        st.session_state.graph = build_graph(engine)
                        if "thread_id" not in st.session_state or not st.session_state.thread_id:
                            import time
                            st.session_state.thread_id = f"ui-{int(time.time())}"
                        st.success("OCR index ready.")
                        st.session_state.processed = True
                    except Exception as e:
                        st.error("Build failed")
                        st.exception(e)
                        return
 
            st.subheader("Timings")
            st.json(st.session_state.get("ocr_timings", {}))
 
            question = st.text_input("Ask a question about your PDFs:")
            if question:
                state = {
                    "messages": [
                        {"role": "system", "content": "You are a financial assistant."},
                        {"role": "user", "content": question},
                    ]
                }
                out = st.session_state.graph.invoke(
                    state,
                    {"configurable": {"thread_id": st.session_state.thread_id}}
                    )
                st.write(out["messages"][-1].content)
    main()######new
# Streamlit runs the script top-level, but keeping this is fine
if __name__ == "__main__":
    print("Running test...")
    pdf_processing("test prompt")


In [94]:
from azure.blob_functions import get_companies

name_map , names = get_companies()

In [101]:
from io import BytesIO
import pandas as pd
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobClient, ContentSettings
from dotenv import load_dotenv, find_dotenv
import os 
import requests
from requests.auth import HTTPBasicAuth

load_dotenv(find_dotenv(), override=True)

# === CONFIG ===
ACCOUNT_URL   = os.getenv("BLOB_ACCOUNT_URL")
AZURE_STORAGE_CONNECTION_STRING=os.getenv("BLOB_AZURE_STORAGE_CONNECTION_STRING")
# BLOB_NAME     = "companieslist/CompaniesHouseList.xlsx"   # e.g., "reports/myfile.xlsx"
UK_API_KEY = os.getenv("UK_API_KEY")

INVALID_CHARS = '<>:"/\\|?*'
def sanitize(s: str) -> str:
    s = s.replace(" ", "_")
    for ch in INVALID_CHARS:
        s = s.replace(ch, "_")
    return s

def companyHouseListAdd(
    CONTAINER='companieslist',
    BLOB_NAME='CompaniesHouseList.xlsx',
    CompanyNumber=None,
    sheet_name='IDs'   # change if your sheet is named differently
):
    if CompanyNumber is None:
        raise ValueError("CompanyNumber is required")

    # 1) Download the Excel
    excel_bytes = get_file_blob(CONTAINER, BLOB_NAME)

    # Try reading the sheet; if file/sheet doesn't exist, start a blank DF
    try:
        df = pd.read_excel(BytesIO(excel_bytes), sheet_name=sheet_name, dtype={'IDS': str, 'NAMES': str})
    except Exception:
        df = pd.DataFrame(columns=['IDS', 'NAMES'])

    # Ensure required columns exist
    for col in ('IDS', 'NAMES'):
        if col not in df.columns:
            df[col] = ""

    # 2) Get the company display name (human-friendly; don't sanitize for Excel)
    url = f"https://api.company-information.service.gov.uk/company/{CompanyNumber}"
    r = requests.get(url, auth=HTTPBasicAuth(UK_API_KEY, ""))
    r.raise_for_status()
    name = r.json().get("company_name", "")
    name = sanitize(name)

    # 3) Upsert: if ID exists, update its name; else append new row
    CompanyNumber = str(CompanyNumber)
    mask = (df['IDS'].astype(str) == CompanyNumber)
    if mask.any():
        df.loc[mask, 'NAMES'] = name
    else:
        df = pd.concat(
            [df, pd.DataFrame({'IDS': [CompanyNumber], 'NAMES': [name]})],
            ignore_index=True
        )

    # Optional: dedupe on IDs keeping the last occurrence
    df = df.drop_duplicates(subset=['IDS'], keep='last')

    # 4) Write back to Excel in-memory
    buf = BytesIO()
    with pd.ExcelWriter(buf, engine="openpyxl") as xw:
        df.to_excel(xw, index=False, sheet_name=sheet_name)
    buf.seek(0)

    return buf.getvalue()


excel_bytes = companyHouseListAdd(CompanyNumber="07584487")

save_to = '/Users/felipesilverio/Documents/GitHub/Azure-OnePager/CompaniesHouseListTest.xlsx'
with open(save_to, "wb") as f:
    f.write(excel_bytes)

In [ ]:
from azure.blob_functions import get_companies
import difflib


def pick_company(user_text):

    name_map, names = get_companies()
    unique_names = list(dict.fromkeys(names))      # de-dupe while keeping order
    reverse_map = {v: k for k, v in name_map.items()}  # clean -> orig
    # normalize user input a bit
    cleaned = user_text.strip().upper()
    
    # try fuzzy match against the official list
    matches = difflib.get_close_matches(
        cleaned,
        names,
        n=1,          # only want the single best
        cutoff=0.6    # 0.0–1.0; raise this if you want to be stricter
    )

    if matches:
        return reverse_map.get(matches[0], matches[0])  # this is the canonical company name
    return None

teste = pick_company(' james donaldson')
teste

'JAMES_DONALDSON_GROUP_LTD'

In [10]:
business_overview_formatting = """
 - This section provides a high-level overview on what the company does, its operations, locations, products, customers and any ongoing debt/financial issues, in a bullet point format consisting of 5-6 bullet points with sentences, using the latest available annual reports/financial statements of the company 
-- Include 1-2 bullet point sentences on what the company does 
-- Include 1 bullet point on the products/services the company offers 
-- Include 1 bullet point on where the company has its operations (e.g. manufacturing facilities, operating plants, offices, customers) 
-- Include 1 bullet point on who are the customers of the company  
-- Include 1 bullet point on stress triggers of the company (e.g., 40% revenue from top 1 customer; high fixed costs; collateral shortfall; aggressive capex; covenant breach; dropping profitability; mass lay-offs etc.) 

- Each bullet must begin with the company name, "The company", or “It”. Make sure each bullet point is a proper sentence, which do not contain any sub-headings, colon or semi-colons 

- Sources to be used for this section:  
-- The bullet points regarding what the company does, its products/services, operations, customers can be sourced from the Primary Activity, Business Review, Introduction or Strategic Report section of the report 
-- The bullet point regarding company’s stress triggers can be sourced from the Business Review or Ongoing Concern or Bank Debt/Borrowings/Creditors section of the report 
-- If any of the above source suggestions does not return results for any part, please scan and check other sections of the reports to see if relevant information can be found 
 
- Notes for this section: 
-- If information for any of the bullet point is not available in the report, do not include that specific bullet point as incorrect information is strictly prohibited 

"""

resp = agent._answer(question=business_overview_formatting, ctx_text=answers)
print(resp['answer'])

- Seaport Topco Limited’s principal activity continues to include the research and development of pharmaceutical instrumentation. [1][2]
- The company is engaged in group-level operations focused on developing pharmaceutical instrumentation through ongoing research and development initiatives. [1][2]
- It offers research and development of pharmaceutical instrumentation as its primary product and service offering. [1][2]
- The company reported a headcount of 418 at Dec-24, up from 405, indicating ongoing operations across the group. [9]
- The company faces near-term liquidity and working capital pressure, with creditors due within one year rising to £64.7m at Dec-24 driven by £7.7m of new loans and a £9.2m net revolving credit facility draw, while cash generated from operating activities declined to £10.4m from £15.3m in FY23. [9]
- It carries substantial longer-term obligations, with creditors due after more than one year at £168.7m at Dec-23, alongside rising trade debtors to £17.0m 

In [ ]:
finance_formatting_2= """ 
Return TWO sections in this exact order:

SECTION 1 — TABLE
- Output a valid Table with header: Metric,FY24,FY23,FY22
- One data row per metric.
- Use "n.a." / "n.m." exactly when unavailable.
- Do NOT add any text before or after the Table in this section.

SECTION 2 — SUMMARY / INTERPRETATION
- After the Table, add a single blank line, then a heading line: Summary / Interpretation
- Provide 3–6 concise bullets explaining the key movements, relationships, and caveats.
- Base all points strictly on the Table values; do not invent numbers.

SECTION 3 - SOURCES
- Point out all the sources used by the original input with the correct number index like [#6], and CITE THE COMPLETE SOURCE like which report it was used, etc.


Formatting example (shape only; values are illustrative):

Metric                  |   FY24                |   FY23            |   FY22
Revenue (Turnover)      |   £576.8m [#2]        |   £81.4m [#6]     |   £32.8m [#5]
Revenue growth % (yoy)  |   +608.6% [#2][#6]    |   +148.0% [#5]    |   n.a.
Gross profit            |   n.a.                |   £48.4m [#3][#6] |   £14.3m [#3]"

Summary / Interpretation
- Brief point 1…
- Brief point 2…
- Brief point 3…
"""

resp = agent._answer(question=finance_formatting_2, ctx_text=answers)
print(resp['answer'])

Metric,FY24,FY23,FY22
"Revenue (Turnover)","£76.8m [#2]","£81.4m [#2]","£32.8m [#2]"
"Revenue growth % (yoy)","-5.7% [#5]","+148.1% [#5]","n.a."
"Gross profit","£43.9m [#3]","£48.4m [#3]","£14.3m [#3]"
"Gross margin %","57.2% [#6]","59.5% [#6]","43.7% [#6]"
"EBITDA","£10.7m [#4]","£20.2m [#4]","-£1.2m [#4]"
"EBITDA margin %","14.0% [#7]","24.9% [#7]","n.m. [#4][#2]"
"Net cash from operating activities","£9.6m [#8]","£15.3m [#8]","-£1.8m [#8]"
"Net working capital (movement)","-£1.2m [#9]","£3.9m [#9]","-£3.0m [#9]"
"CFO excl. Net working capital","£10.8m [#8]","£11.4m [#8]","£1.2m [#8]"
"Capex (tangible + intangible)","£2.9m [#10]","£7.2m [#10]","£0.9m [#10]"
"Other cash flow from investing activities","£0.2m [#11]","-£0.8m [#11]","-£638.1m [#11]"
"CFADS (CFO + CFI)","£6.9m [#1]","£7.3m [#8][#11]","-£640.8m [#8][#11]"
"Financing cash flow — Issue of shares","£1.5m [#12]","£0.4m [#12]","£484.2m [#12]"
"Financing cash flow — Draw down of bank loans","£17.9m [#12]","£12.1m [#12]","£187.3m

In [ ]:
stakeholders_formatting = """ 
Return TWO sections in this exact order:

SECTION 1 — CSV TABLE
- Output a valid CSV with header: Metric,Shareholders
- One data row per metric.
- Use "n.a." / "n.m." exactly when unavailable.
- CSV rules:
  * Separate fields with commas only (no extra spaces around commas).
  * Wrap any field that contains commas, brackets, percent signs, currency symbols, or spaces in double quotes.
  * Escape any internal double quotes by doubling them.
- Do NOT wrap the CSV in code fences.
- Do NOT add any text before or after the CSV in this section.

SECTION 2 — SUMMARY / INTERPRETATION
- After the CSV, add a single blank line, then a heading line: Summary / Interpretation
- Provide 3–6 concise bullets explaining the key movements, relationships, and caveats.
- Base all points strictly on the CSV values; do not invent numbers.

SECTION 3 - SOURCES
- Point out all the sources used by the original input with the correct number index like [#6], and CITE THE COMPLETE SOURCE like which report it was used, etc.


Formatting example (shape only; values are illustrative):

Metric,Shareholders
"Shareholders", "Scott"
"Management","n.a."
"Lenders","Maria"
"Auditors","James"
"Advisors","n.a."

Summary / Interpretation
- Brief point 1…
- Brief point 2…
- Brief point 3…
"""

resp = agent._answer(question=stakeholders_formatting, ctx_text=answers)
print(resp['answer'])

'Metric,Shareholders\n"Shareholders","Immediate parent: EQT Jupiter Luxco S.A.R.L (Luxembourg); Ultimate parent: n.a."\n"Management","Chairman: n.a.; CEO: n.a.; CFO: n.a.; Directors (FY24): M Bauer; R Diggelmann (resigned Dec-24); P Dowdy (resigned Feb-25); J Feldman; R Friel (resigned Mar-25); K Murphy; D Newble (resigned Jul-24); A Thorburn; R Walton (appointed Jul-24)"\n"Lenders","n.a."\n"Auditors","n.a."\n"Advisors","Facility agent: Kroll Agency Services Limited; Financial advisor: n.a.; Legal advisor: n.a.; Bankers: n.a."\n\nSummary / Interpretation\n- Ownership is clearly identified at the immediate parent level (EQT Jupiter Luxco S.A.R.L), while the ultimate parent is n.a., indicating Seaport Topco Limited is the head of the consolidation group.\n- Key executive roles (Chairman/CEO/CFO) are n.a., but a detailed FY24 directors list is provided, including multiple resignations and one appointment.\n- Lender names are n.a., so external creditor identification is not available from 

In [59]:
print(resp['answer'])

Metric,Shareholders
"Shareholders","Immediate parent: EQT Jupiter Luxco S.A.R.L (Luxembourg); Ultimate parent: n.a."
"Management","Chairman: n.a.; CEO: n.a.; CFO: n.a.; Directors (FY24): M Bauer; R Diggelmann (resigned Dec-24); P Dowdy (resigned Feb-25); J Feldman; R Friel (resigned Mar-25); K Murphy; D Newble (resigned Jul-24); A Thorburn; R Walton (appointed Jul-24)"
"Lenders","n.a."
"Auditors","n.a."
"Advisors","Facility agent: Kroll Agency Services Limited; Financial advisor: n.a.; Legal advisor: n.a.; Bankers: n.a."

Summary / Interpretation
- Ownership is clearly identified at the immediate parent level (EQT Jupiter Luxco S.A.R.L), while the ultimate parent is n.a., indicating Seaport Topco Limited is the head of the consolidation group.
- Key executive roles (Chairman/CEO/CFO) are n.a., but a detailed FY24 directors list is provided, including multiple resignations and one appointment.
- Lender names are n.a., so external creditor identification is not available from the provide

In [11]:
import io, re
from docx import Document
from docx.enum.text import WD_BREAK

# =========================
# 0) Your full GPT response
# =========================
gpt_output = r"""
- Seaport Topco Limited’s principal activity continues to include the research and development of pharmaceutical instrumentation. [1][2]
- The company is engaged in group-level operations focused on developing pharmaceutical instrumentation through ongoing research and development initiatives. [1][2]
- It offers research and development of pharmaceutical instrumentation as its primary product and service offering. [1][2]
- The company reported a headcount of 418 at Dec-24, up from 405, indicating ongoing operations across the group. [9]
- The company faces near-term liquidity and working capital pressure, with creditors due within one year rising to £64.7m at Dec-24 driven by £7.7m of new loans and a £9.2m net revolving credit facility draw, while cash generated from operating activities declined to £10.4m from £15.3m in FY23. [9]
- It carries substantial longer-term obligations, with creditors due after more than one year at £168.7m at Dec-23, alongside rising trade debtors to £17.0m and reduced stock to £13.2m at Dec-24, indicating cash conversion and balance sheet pressures. [7][8][9]

Sources:
- [1][2] Annual Report (FY23), Strategic/Directors’ Report, p.5 — Principal activity.
- [9] Annual Report (FY24), Group Strategic Report (pp.1–5; exact page n.a.) — Review of the business and future developments, working capital and liquidity movements.
- [7][8] Annual Report (FY23), Notes 21–22, Consolidated Statement of Financial Position, p.14 — Creditors due within one year and after more than one year.
""".strip("\n")

# =========================
# 1) Open DOCX
# =========================
doc_path = "/Users/felipesilverio/Documents/GitHub/Azure-OnePager/CompanyProfile2.docx"
doc = Document(doc_path)

PLACEHOLDER = "[INSERT BUSINESS OVERVIEW]"

def set_paragraph_multiline(paragraph, text: str):
    """Replace a paragraph's text with multi-line content, preserving line breaks."""
    # clear existing runs
    for run in paragraph.runs:
        run.text = ""
    # write lines with explicit line breaks
    lines = (text or "").splitlines()
    if not lines:
        return
    paragraph.add_run(lines[0])
    for ln in lines[1:]:
        r = paragraph.add_run()
        r.add_break(WD_BREAK.LINE)
        paragraph.add_run(ln)

def replace_placeholder(document: Document, placeholder: str, new_text: str) -> bool:
    """Find placeholder in paragraphs/cells and replace it with new_text (multiline)."""
    # plain paragraphs
    for p in document.paragraphs:
        if placeholder in p.text:
            set_paragraph_multiline(p, new_text)
            return True
    # inside tables
    for tbl in document.tables:
        for row in tbl.rows:
            for cell in row.cells:
                for p in cell.paragraphs:
                    if placeholder in p.text:
                        set_paragraph_multiline(p, new_text)
                        return True
    return False

# =========================
# 2) Replace the placeholder
# =========================
ok = replace_placeholder(doc, PLACEHOLDER, gpt_output)
if not ok:
    print(f"WARNING: placeholder not found: {PLACEHOLDER}")

# =========================
# 3) Save
# =========================
out_path = "/Users/felipesilverio/Documents/GitHub/Azure-OnePager/CompanyProfile2.docx"
doc.save(out_path)
print(f"Updated document written to: {out_path}")


Updated document written to: /Users/felipesilverio/Documents/GitHub/Azure-OnePager/CompanyProfile2.docx


In [13]:
import io, re
import pandas as pd
from docx import Document
from docx.enum.text import WD_BREAK

# =========================
# 0) Your full GPT response
# =========================
gpt_output = r"""
Metric,FY24,FY23,FY22
"Facility Name","n.a.","Facility B1 (EUR term loan); Facility B2 (USD term loan); Revolving Credit Facility; Delayed Drawdown Facility [#2][#3]","Revolving Credit Facility; Delayed Drawdown Facility [#3]"
"Interest Rate","n.a.","Euribor + 6.25%; Term SOFR + 6.25% [#2][#3]","n.a."
Maturity,"n.a.","Aug-29 [#2][#3]","n.a."
"Adjusted EBITDA","n.a.","£17.9m [#2][#3]","£10.6m [#2][#3]"
"Cash (Closing Cash)","n.a.","£11.8m [#2][#3]","n.a."
"Net Debt","n.a.","£171.9m [#2][#3]","n.a."
Liquidity,"n.a.","£25.8m [#2][#3]","n.a."
"Leverage (Net Debt/EBITDA)","n.a.","9.6x [#2][#3]","n.a."
"Facility B1 outstanding (GBP)","n.a.","£36.0m [#2][#3]","n.a."
"Facility B2 outstanding (GBP)","n.a.","£135.0m [#2][#3]","n.a."
"RCF drawn","n.a.","£16.0m [#2][#3]","£16.0m [#3]"
"RCF facility size","n.a.","£30.0m [#2][#3]","£30.0m [#3]"
"Delayed Drawdown Facility size","n.a.","£75.0m [#2][#3]","£75.0m [#3]"
"Bank loans due after >5 years","n.a.","£168.7m [#2][#3]","n.a."
"Bank loans due within 1 year","n.a.","£14.7m [#2][#3]","n.a."
"Bank loans + RCF outstanding (excl. leases)","n.a.","£187.0m [#2][#3]","n.a."

Summary / Interpretation
- FY23 leverage is high at 9.6x, based on £171.9m net debt and £17.9m Adjusted EBITDA.
- FY23 facility mix is dominated by term loans (Facility B1 ~£36.0m and B2 ~£135.0m) maturing in Aug-29, plus a £30.0m RCF (of which £16.0m was drawn) and a £75.0m delayed draw facility.
- FY23 liquidity appears modest at £25.8m, combining £11.8m closing cash with remaining headroom on the £30.0m RCF (drawn £16.0m).
- Maturity profile in FY23 is back-ended: £168.7m due after >5 years versus £14.7m due within 1 year; both term facilities mature in Aug-29.
- Total FY23 bank loans + RCF outstanding (excl. leases) sums to £187.0m, indicating a sizeable secured debt stack.
- FY24 disclosures are not available in the provided excerpts, and FY22 data is limited beyond RCF details.

Sources
- [#2] Seaport Topco Limited Annual Report (file date 25-Sep-24), pp. 8, 45, 52 — p.8 Adjusted EBITDA (£17.9m FY23; £10.6m FY22); p.45 Loans note (Facility B1/B2 amounts, Aug-29 maturities, interest margins; RCF £30.0m and ~£16.0m drawn; £75.0m delayed draw facility; bank loans due >5 years £168.7m and within 1 year £14.7m); p.52 Net debt analysis (Net Debt £171.9m; closing cash £11.8m). Link: https://aiprojectteneo.blob.core.windows.net/companieshouselinglefile/14171962/SEAPORT_TOPCO_LIMITED_AA_annualReport_2024-09-25_0.pdf
- [#3] Seaport Topco Limited Annual Report (file date 25-Sep-24), pp. 8, 36, 45, 52 — corroborates Adjusted EBITDA figures; Going concern (p.36) notes RCF drawn £16.0m at Dec-22; Loans note (p.45) for facility sizes/draws and maturities; Net debt analysis (p.52) for Net Debt and closing cash. Link: https://aiprojectteneo.blob.core.windows.net/companieshousinglefile/14171962/SEAPORT_TOPCO_LIMITED_AA_annualReport_2024-09-25_1.pdf
"""

# =========================
# 1) Extract CSV + SUMMARY  (KEEP sources)
# =========================
parts = gpt_output.split("\n\nSummary / Interpretation", 1)
csv_block = parts[0].strip()

start = csv_block.find("Metric,")
if start == -1:
    raise ValueError("CSV header 'Metric,' not found in model output.")
csv_block = csv_block[start:]

summary_text = ""
if len(parts) > 1:
    summary_text = "Summary / Interpretation" + parts[1].rstrip()

# =========================
# 2) Parse CSV to DataFrame
# =========================
df = pd.read_csv(io.StringIO(csv_block))
expected_cols = {"Metric","FY24"}
if not expected_cols.issubset(df.columns):
    raise ValueError(f"Capital Structure CSV columns missing. Got: {list(df.columns)}")

csv_rows = {
    str(df.at[i, "Metric"]).strip(): {
        "FY24": str(df.at[i, "FY24"])
    }
    for i in range(len(df))
}

# =========================
# 3) Open DOCX, locate the Capital Structure table
# =========================
doc_path = "/Users/felipesilverio/Documents/GitHub/Azure-OnePager/CompanyProfile2.docx"
doc = Document(doc_path)

# ----------------- helpers -----------------
def norm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+", "", (s or "").lower())

def tokens(s: str) -> set:
    return set(re.findall(r"[a-z0-9]+", (s or "").lower()))

def jaccard(a: str, b: str) -> float:
    ta, tb = tokens(a), tokens(b)
    if not ta or not tb:
        return 0.0
    inter = len(ta & tb)
    union = len(ta | tb)
    return inter / union if union else 0.0

def find_cap_struct_table(document: Document):
    # 1) after 'Capital Structure' heading
    found_heading = False
    body = document._element.body
    for child in body.iterchildren():
        tag = child.tag.rsplit("}", 1)[-1]
        if tag == "p":
            p_text = "".join(t.text for t in child.iter()
                             if t.tag.rsplit("}",1)[-1] == "t").strip()
            if norm(p_text) == "capitalstructure":
                found_heading = True
        elif tag == "tbl" and found_heading:
            from docx.table import Table
            return Table(child, document)

    # 2) heuristic by content
    for tbl in document.tables:
        row_texts = [" ".join(c.text for c in r.cells) for r in tbl.rows]
        joined = " ".join(row_texts)
        if all(x in norm(joined) for x in ["ebitda","leverage"]):
            return tbl
    return None

table = find_cap_struct_table(doc)
if table is None:
    raise RuntimeError("Could not locate the 'Capital Structure' table.")

# Identify FY columns
def find_fy_cols(tbl):
    for r_i in range(min(2, len(tbl.rows))):
        labels = [norm(c.text) for c in tbl.rows[r_i].cells]
        loc = {}
        for idx, txt in enumerate(labels):
            if txt == "fy24": loc["FY24"] = idx
        if {"FY24"}.issubset(loc.keys()):
            return loc["FY24"]
    if len(tbl.rows[0].cells) >= 4:
        return 1, 2, 3
    raise RuntimeError("Could not determine FY columns in Capital Structure table.")

col_FY24 = find_fy_cols(table)

# Build row index from first column (labels)
doc_row_index = {}
doc_row_labels = {}  # norm_label -> raw label (for debug)
for r_idx, row in enumerate(table.rows):
    if not row.cells:
        continue
    label_raw = row.cells[0].text.strip()
    if label_raw:
        key = norm(label_raw)
        doc_row_index[key] = r_idx
        doc_row_labels[key] = label_raw

# =========================
# 4) Mapping (CSV -> DOC label), with synonyms & typo tolerance
# =========================
def keynorm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+", "", (s or "").lower())

# Add broad synonyms, incl. likely template wordings
metric_to_doc_syns = {
    keynorm("Facility Name"): [
        "Facility Name", "Name of the Facility", "Facility", "Facilities", "Facility Names",
        "Name of Facility"
    ],
    keynorm("Interest Rate"): ["Interest Rate", "Interst Rate", "Rate", "Interest"],
    keynorm("Interst Rate"):  ["Interest Rate", "Interst Rate", "Rate", "Interest"],
    keynorm("Maturity"): ["Maturity", "Final Maturity", "Maturities", "Maturity Date"],
    keynorm("Adjusted EBITDA"): ["EBITDA", "Adjusted EBITDA"],
    keynorm("Cash (Closing Cash)"): [
        "Cash (Closing Cash)", "Cash (Closing cash)", "Closing Cash",
        "Cash", "Cash and cash equivalents", "Cash & cash equivalents"
    ],
    keynorm("Net Debt"): ["Net External Debt", "Net Debt"],
    keynorm("Liquidity"): ["Liquidity"],
    keynorm("Leverage (Net Debt/EBITDA)"): ["Leverage"],
    keynorm("Leverage (Net Debt / EBITDA)"): ["Leverage"],
    keynorm("Bank loans + RCF outstanding (excl. leases)"): ["Gross External Debt", "Total External Debt"],

    keynorm("Facility B1 outstanding (GBP)"): ["Amount Outstanding"],
    keynorm("Facility B2 outstanding (GBP)"): ["Amount Outstanding"],
    keynorm("RCF drawn"): ["Amount Outstanding"],

    keynorm("RCF facility size"): [None],
    keynorm("Delayed Drawdown Facility size"): [None],
    keynorm("Bank loans due after >5 years"): [None],
    keynorm("Bank loans due within 1 year"): [None],
}

def smart_lookup_row_index(label_candidates):
    """
    Resolve to the best row index by:
      1) Exact normalized match
      2) Contains match (both directions)
      3) Token Jaccard similarity >= 0.5
    Returns row_index or None.
    """
    cand_norms = [norm(c) for c in label_candidates if c]

    # 1) exact
    for cn in cand_norms:
        if cn in doc_row_index:
            return doc_row_index[cn]

    # 2) contains (prefer the longest doc label match)
    best_idx = None
    best_len = -1
    for cn in cand_norms:
        for dl_norm, idx in doc_row_index.items():
            if cn and (cn in dl_norm or dl_norm in cn):
                if len(dl_norm) > best_len:
                    best_idx, best_len = idx, len(dl_norm)
    if best_idx is not None:
        return best_idx

    # 3) token overlap
    best_idx = None
    best_score = 0.0
    for cn in cand_norms:
        for dl_norm, idx in doc_row_index.items():
            score = jaccard(cn, dl_norm)
            if score >= 0.5 and score > best_score:
                best_idx, best_score = idx, score
    return best_idx

# =========================
# 5) Populate the table (incl. Facility Name / Interest Rate / Maturity)
#    and aggregate facility amounts into 'Amount Outstanding'
# =========================
agg_amount = {"FY24": []}

def maybe_append(prefix, v):
    v = (v or "").strip()
    if not v or v.lower() == "n.a.":
        return None
    return f"{prefix}: {v}"

unmapped_metrics = []

for csv_metric, years in csv_rows.items():
    mkey = keynorm(csv_metric)
    syns = metric_to_doc_syns.get(mkey, [csv_metric])

    # aggregated targets
    if any((s and norm(s) == norm("Amount Outstanding")) for s in syns if s):
        if mkey == keynorm("Facility B1 outstanding (GBP)"):
            for fy in ("FY24",):
                s = maybe_append("B1", years[fy]);  agg_amount[fy].append(s) if s else None
        elif mkey == keynorm("Facility B2 outstanding (GBP)"):
            for fy in ("FY24",):
                s = maybe_append("B2", years[fy]);  agg_amount[fy].append(s) if s else None
        elif mkey == keynorm("RCF drawn"):
            for fy in ("FY24",):
                s = maybe_append("RCF drawn", years[fy]);  agg_amount[fy].append(s) if s else None
        continue

    r_idx = smart_lookup_row_index(syns)
    if r_idx is None:
        unmapped_metrics.append(csv_metric)
        continue

    row = table.rows[r_idx]
    row.cells[col_FY24].text = years["FY24"]

# Aggregated 'Amount Outstanding'
amount_row_idx = smart_lookup_row_index(["Amount Outstanding"])
if amount_row_idx is not None:
    row = table.rows[amount_row_idx]
    row.cells[col_FY24].text = "; ".join(agg_amount["FY24"]) if agg_amount["FY24"] else "n.a."

# =========================
# 6) Insert the full SUMMARY (including Sources) — no duplicates
# =========================
PLACEHOLDER = "[INSERT CAPITAL STRUCTURE SUMMARY]"
HEADING_TEXT = "Summary / Interpretation"

def set_paragraph_multiline(paragraph, text: str):
    for run in paragraph.runs:  # clear
        run.text = ""
    lines = (text or "").splitlines()
    if not lines:
        return
    paragraph.add_run(lines[0])
    for ln in lines[1:]:
        paragraph.add_run().add_break(WD_BREAK.LINE)
        paragraph.add_run(ln)

def replace_placeholder(document: Document, placeholder: str, new_text: str) -> bool:
    # paragraphs
    for p in document.paragraphs:
        if placeholder in p.text:
            set_paragraph_multiline(p, new_text)
            return True
    # tables
    for tbl in document.tables:
        for row in tbl.rows:
            for cell in row.cells:
                for p in cell.paragraphs:
                    if placeholder in p.text:
                        set_paragraph_multiline(p, new_text)
                        return True
    return False

def find_all_summary_paragraphs(document: Document, heading_text: str):
    anchors = []
    for p in document.paragraphs:
        if heading_text in p.text:
            anchors.append(p)
    for tbl in document.tables:
        for row in tbl.rows:
            for cell in row.cells:
                for p in cell.paragraphs:
                    if heading_text in p.text:
                        anchors.append(p)
    return anchors

insert_done = False
if summary_text:
    insert_done = replace_placeholder(doc, PLACEHOLDER, summary_text)

if summary_text and not insert_done:
    anchors = find_all_summary_paragraphs(doc, HEADING_TEXT)
    if anchors:
        # overwrite first and remove extras
        set_paragraph_multiline(anchors[0], summary_text)
        for dup in anchors[1:]:
            dup._element.getparent().remove(dup._element)
    else:
        p = doc.add_paragraph()
        set_paragraph_multiline(p, summary_text)
        # ensure no accidental multiples
        anchors = find_all_summary_paragraphs(doc, HEADING_TEXT)
        for dup in anchors[1:]:
            dup._element.getparent().remove(dup._element)

# =========================
# 7) Save + (optional) debug
# =========================
out_path = "/Users/felipesilverio/Documents/GitHub/Azure-OnePager/CompanyProfile2.docx"
doc.save(out_path)
print(f"Updated document written to: {out_path}")

if unmapped_metrics:
    print("NOTE — CSV metrics that couldn't be matched to any row (check your template labels):")
    for m in unmapped_metrics:
        print(" -", m)


Updated document written to: /Users/felipesilverio/Documents/GitHub/Azure-OnePager/CompanyProfile2.docx
NOTE — CSV metrics that couldn't be matched to any row (check your template labels):
 - RCF facility size
 - Delayed Drawdown Facility size
 - Bank loans due after >5 years
 - Bank loans due within 1 year


In [10]:
import io, re
import pandas as pd
from docx import Document
from docx.enum.text import WD_BREAK

# =========================
# 0) Your full GPT response
# =========================
gpt_output = r"""
Metric,Shareholders
"Shareholders",n.a.
"Management","Directors (FY24): M Bauer; R Diggelmann — resigned Dec-24; P Dowdy — resigned Feb-25; J Feldman; R Friel — resigned Mar-25; K Murphy; D Newble — resigned Jul-24; A Thorburn; R Walton — appointed Jul-24"
"Lenders",n.a.
"Auditors","Grant Thornton UK LLP (Statutory Auditor); Stephen Wyborn (Senior Statutory Auditor)"
"Advisors","Facility agent: Kroll Agency Services Limited; Bankers: n.a.; Solicitors: n.a.; Financial advisor: n.a."

Summary / Interpretation
- Shareholder information is n.a., indicating no disclosed immediate or ultimate parent in the provided filings.
- Management is represented by the FY24 directors list; specific Chairman/CEO/CFO titles are not provided.
- Lenders are n.a., suggesting no disclosed bank facilities/borrowings in the available excerpts.
- Auditors are identified (Grant Thornton UK LLP; Senior Statutory Auditor Stephen Wyborn), while most other advisors are n.a. except the facility agent (Kroll Agency Services Limited).
- Advisor disclosure is limited (bankers/solicitors/financial advisor n.a.), constraining visibility into counterparties.

SECTION 3 - SOURCES
- [#1] Seaport Topco Limited AA Annual Report (published Sep-25), Directors’ Report for the year ended 31 Dec-24, p.12. Link: https://aiprojectteneo.blob.core.windows.net/companieshousesinglefile/14171962/SEAPORT_TOPCO_LIMITED_AA_annualReport_2025-09-30_0.pdf — used for the “Management” (directors) row.
- [#2] Seaport Topco Limited AA Annual Report (2024-09-25) – provided excerpt indicating Company Information page not available — supports n.a. for certain advisor details.
- [#3] Seaport Topco Limited annual report (FY24), Notes to the Financial Statements – Accounting policies excerpt (page n.a.) — used to check terminology (bank loans/borrowings); lenders n.a. and facility agent reference noted elsewhere.
- [#4] Seaport Topco Limited Annual Report 2024, Independent Auditors’ Report signature block, file: SEAPORT_TOPCO_LIMITED_AA_annualReport_2024-09-25_1.pdf (26 pages), signed Apr-24 — used for “Auditors” identification and to support shareholder n.a.
- [#5] Seaport Topco Limited Annual Report (published Sep-24), Notes to the Financial Statements, Note 16: Fixed asset investments – Indirect subsidiary undertakings (page n.a.) — used to support “Shareholders” n.a. (no parent/ultimate controlling party disclosed).
- [#7] Seaport Topco Limited Annual Report 2024, Independent Auditors’ Report signature block, file: SEAPORT_TOPCO_LIMITED_AA_annualReport_2024-09-25_0.pdf (26 pages), signed Apr-24 — corroborates “Auditors” identification.
"""

# =========================
# 1) Extract CSV + SUMMARY
# =========================
parts = gpt_output.split("\n\nSummary / Interpretation", 1)
csv_block = parts[0].strip()

start = csv_block.find("Metric,")
if start == -1:
    raise ValueError("CSV header 'Metric,' not found in model output.")
csv_block = csv_block[start:]

summary_text = ""
if len(parts) > 1:
    summary_text = "Summary / Interpretation" + parts[1].rstrip()

# =========================
# 2) Parse CSV to DataFrame
# =========================
df = pd.read_csv(io.StringIO(csv_block))
expected_cols = {"Metric", "Shareholders"}
if not expected_cols.issubset(df.columns):
    raise ValueError(f"Key Stakeholders CSV columns missing. Got: {list(df.columns)}")

# Dict: metric -> value (right column)
ks_rows = {
    str(df.at[i, "Metric"]).strip(): str(df.at[i, "Shareholders"]).strip()
    for i in range(len(df))
}

# ==============================================
# 3) Open DOCX, find the "Key Stakeholders" table
# ==============================================
doc_path = "/Users/felipesilverio/Documents/GitHub/Azure-OnePager/CompanyProfile (1).docx"
doc = Document(doc_path)

def norm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+", "", (s or "").lower())

def find_ks_table(document: Document):
    # Prefer a table whose header has both "Title" and "Occupants"
    for tbl in document.tables:
        if not tbl.rows:
            continue
        header = [norm(c.text) for c in tbl.rows[0].cells]
        if "title" in header and "occupants" in header:
            return tbl

    # Fallback: first table after the "Key Stakeholders" heading
    found_heading = False
    body = document._element.body
    for child in body.iterchildren():
        tag = child.tag.rsplit("}", 1)[-1]
        if tag == "p":
            p_text = "".join(t.text for t in child.iter()
                             if t.tag.rsplit("}",1)[-1] == "t").strip()
            if norm(p_text) == "keystakeholders":
                found_heading = True
        elif tag == "tbl" and found_heading:
            from docx.table import Table
            return Table(child, document)
    return None

ks_table = find_ks_table(doc)
if ks_table is None:
    raise RuntimeError("Could not locate the 'Key Stakeholders' table (Title | Occupants).")

# ==============================================
# 4) Detect columns: label = "Title", value = "Occupants"
# ==============================================
def get_title_and_occupants_cols(tbl):
    # Defaults for a 2-col layout
    label_col, value_col = 0, 1

    if tbl.rows:
        header_norm = [norm(c.text) for c in tbl.rows[0].cells]
        if "title" in header_norm:
            label_col = header_norm.index("title")
        if "occupants" in header_norm:
            value_col = header_norm.index("occupants")

    # Ensure they are different; if not, force value_col to the other col
    if value_col == label_col and len(tbl.rows[0].cells) >= 2:
        value_col = 1 if label_col == 0 else 0
    return label_col, value_col

label_col, value_col = get_title_and_occupants_cols(ks_table)

# ==============================================
# 5) Build row index using the Title (label) column
# ==============================================
row_index = {}
for r_idx, row in enumerate(ks_table.rows):
    if not row.cells:
        continue
    # Skip header
    if r_idx == 0:
        continue
    label_text = row.cells[label_col].text.strip()
    if label_text:
        row_index[norm(label_text)] = r_idx

# ==============================================
# 6) Map CSV metric names → Title labels
# ==============================================
def keynorm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+", "", (s or "").lower())

metric_to_title = {
    keynorm("Shareholders"): "Shareholders",
    keynorm("Management"):  "Management",
    keynorm("Lenders"):     "Lenders",
    keynorm("Auditors"):    "Auditors",
    keynorm("Advisors"):    "Advisors",
}

# ==============================================
# 7) Populate the Occupants column ONLY
# ==============================================
not_found = []
for metric, value in ks_rows.items():
    title_label = metric_to_title.get(keynorm(metric), metric)
    r_idx = row_index.get(norm(title_label))
    if r_idx is None:
        not_found.append(metric)
        continue
    # write into Occupants cell
    ks_table.rows[r_idx].cells[value_col].text = value

if not_found:
    print("WARNING — missing rows for:", ", ".join(not_found))

# ==============================================
# 8) Replace the placeholder with the KS SUMMARY text (optional)
#     (placeholder: [INSERT KEY STAKEHOLDERS SUMMARY])
# ==============================================
KS_PLACEHOLDER = "[INSERT KEY STAKEHOLDERS SUMMARY]"

def set_paragraph_multiline(paragraph, text: str):
    for run in paragraph.runs:
        run.text = ""
    lines = (text or "").splitlines()
    if not lines:
        return
    paragraph.add_run(lines[0])
    for ln in lines[1:]:
        paragraph.add_run().add_break(WD_BREAK.LINE)
        paragraph.add_run(ln)

def replace_placeholder(document: Document, placeholder: str, new_text: str) -> bool:
    # paragraphs
    for p in document.paragraphs:
        if placeholder in p.text:
            set_paragraph_multiline(p, new_text)
            return True
    # cells
    for tbl in document.tables:
        for row in tbl.rows:
            for cell in row.cells:
                for p in cell.paragraphs:
                    if placeholder in p.text:
                        set_paragraph_multiline(p, new_text)
                        return True
    return False

if summary_text:
    ok = replace_placeholder(doc, KS_PLACEHOLDER, summary_text)
    if not ok:
        print("NOTE: placeholder not found:", KS_PLACEHOLDER)

# ==============================================
# 9) Save
# ==============================================
out_path = "/Users/felipesilverio/Documents/GitHub/Azure-OnePager/CompanyProfile2.docx"
doc.save(out_path)
print(f"Updated document written to: {out_path}")


Updated document written to: /Users/felipesilverio/Documents/GitHub/Azure-OnePager/CompanyProfile2.docx


In [12]:
import io, re
import pandas as pd
from docx import Document
from docx.enum.text import WD_BREAK

# =========================
# 0) Your full GPT response
# =========================
gpt_output = finance_output

# =========================
# 1) Extract CSV + SUMMARY
# =========================
# Split off the summary block (everything after the blank line + heading)
parts = gpt_output.split("\n\nSummary / Interpretation", 1)
csv_block = parts[0].strip()

# Ensure we start at the CSV header
start = csv_block.find("Metric,")
if start == -1:
    raise ValueError("CSV header 'Metric,' not found in model output.")
csv_block = csv_block[start:]

# Summary text (keep heading + bullets if present)
summary_text = ""
if len(parts) > 1:
    summary_text = "Summary / Interpretation" + parts[1].rstrip()

# =========================
# 2) Parse CSV to DataFrame
# =========================
df = pd.read_csv(io.StringIO(csv_block))
expected_cols = {"Metric","FY24","FY23","FY22"}
if not expected_cols.issubset(df.columns):
    raise ValueError(f"CSV columns missing. Got: {list(df.columns)}")

# Dict: metric -> {FY24, FY23, FY22}
csv_rows = {
    str(df.at[i, "Metric"]).strip(): {
        "FY24": df.at[i, "FY24"],
        "FY23": df.at[i, "FY23"],
        "FY22": df.at[i, "FY22"],
    }
    for i in range(len(df))
}

# ==============================================
# 3) Open DOCX, find the Financial Performance table
# ==============================================
doc_path = "/Users/felipesilverio/Documents/GitHub/Azure-OnePager/CompanyProfile (1).docx"
doc = Document(doc_path)

def norm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+", "", (s or "").lower())

def find_fin_perf_table(document: Document):
    # Heuristic 1: find a table whose header row contains FY24/FY23/FY22
    for tbl in document.tables:
        if len(tbl.rows):
            header = " ".join(c.text for c in tbl.rows[0].cells)
            if all(x in norm(header) for x in ["fy24","fy23","fy22"]):
                return tbl
    # Heuristic 2: first table after a paragraph exactly "Financial Performance"
    found_heading = False
    body = document._element.body
    for child in body.iterchildren():
        tag = child.tag.rsplit("}", 1)[-1]
        if tag == "p":
            p_text = "".join(t.text for t in child.iter() if t.tag.rsplit("}",1)[-1] == "t").strip()
            if norm(p_text) == "financialperformance":
                found_heading = True
        elif tag == "tbl" and found_heading:
            from docx.table import Table
            return Table(child, document)
    return None

table = find_fin_perf_table(doc)
if table is None:
    raise RuntimeError("Could not locate the 'Financial Performance' table.")

# ==============================================
# 4) Map CSV metric names → rows in the template
# ==============================================
# --- Normalizers (use same rule for CSV and map keys) ---
def keynorm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+", "", (s or "").lower())

# Map CSV metric -> DOC row label (left column text in your template)
# Use normalized keys on the left so variations in spaces/slashes/plus signs won't break it.
metric_map_norm = {
    keynorm("Revenue (Turnover)"): "Revenue",
    keynorm("Revenue growth % (yoy)"): "Revenue Growth",
    keynorm("Gross profit"): "Gross Profit",
    keynorm("Gross margin %"): "Gross Margin",
    keynorm("EBITDA"): "EBITDA",
    keynorm("EBITDA margin %"): "EBITDA Margin",
    keynorm("Adjusted EBITDA"): "Adjusted EBITDA",

    # >>> The ones you said aren't populating <<<
    keynorm("Capex (tangible+intangible)"): "CAPEX",  # DOC label
    keynorm("Capex (tangible + intangible)"): "CAPEX",  # alt form (spaces)
    keynorm("Net Working Capital (change)"): "NET_WORK",
    keynorm("Cash Flow from Financing Activities (net)"): "Cash Flow from Financing Activities",
    keynorm("Net cash from financing activities"): "CASH_FINAN",  # alt wording
    keynorm("Total Debt (external)"): "TOTAL_DEBT",
    keynorm("Total debt (bank + lease liabilities)"): "TOTAL_DEBT",  # alt wording
    keynorm("Leverage (Net Debt/EBITDA)"): "LEVERAGE",
    keynorm("Leverage (Net Debt / EBITDA)"): "LEVERAGE",

    # Other cash flow items you have
    keynorm("Net cash from operating activities"): "Cash Flow from Operating Activities",
    keynorm("Net Working Capital (change)"): "Net Working Capital",  # duplicate on purpose (case variant)
    keynorm("Operating cash flow excl. NWC"): "Cash Flow from Operating Activities excl. Net Working Capital",
    keynorm("Other Cash Flow from Investing Activities"): "Other Cash Flow from Investing Activities",
    keynorm("Net cash from investing activities"): "Net Cash Flow from Investing Activities",
    keynorm("CFADS"): "CFADS",
    keynorm("Opening Cash"): "Opening Cash",
    keynorm("Change in Cash"): "Change in Cash",
    keynorm("Closing Cash"): "Closing Cash",
    keynorm("Bank loans outstanding"): "Total Debt",  # if your template has separate row, adjust
    keynorm("Net Debt"): "Net Debt",
}

# Build lookup of row labels in the DOCX (first column). You already did:
doc_row_index = {}
for r_idx, row in enumerate(table.rows):
    label = row.cells[0].text.strip()
    if label:
        doc_row_index[norm(label)] = r_idx  # norm = your doc normalizer (same idea as keynorm)

# Identify FY columns in header row (handles "FY 24" vs "FY24")
header_norm = [norm(c.text) for c in table.rows[0].cells]
try:
    col_FY24 = header_norm.index("fy24")
    col_FY23 = header_norm.index("fy23")
    col_FY22 = header_norm.index("fy22")
except ValueError:
    # If header is the second row in your template, try that
    header_norm = [norm(c.text) for c in table.rows[1].cells]
    col_FY24 = header_norm.index("fy24")
    col_FY23 = header_norm.index("fy23")
    col_FY22 = header_norm.index("fy22")


# Populate table
not_found = []

for csv_metric, years in csv_rows.items():
    # Find the DOC row label using normalized CSV metric text
    target_label = metric_map_norm.get(keynorm(csv_metric), csv_metric)  # fall back to same text
    r_idx = doc_row_index.get(norm(target_label))  # norm() is your existing doc normalizer
    if r_idx is None:
        not_found.append(csv_metric)
        continue

    row = table.rows[r_idx]
    row.cells[col_FY24].text = str(years["FY24"])
    row.cells[col_FY23].text = str(years["FY23"])
    row.cells[col_FY22].text = str(years["FY22"])

# ==============================================
# 5) Populate the table from CSV
# ==============================================
not_found = []

for csv_metric, years in csv_rows.items():
    target_label = metric_map_norm.get(csv_metric, csv_metric)
    r_idx = doc_row_index.get(norm(target_label))
    if r_idx is None:
        not_found.append(csv_metric)
        continue

    row = table.rows[r_idx]
    row.cells[col_FY24].text = str(years["FY24"])
    row.cells[col_FY23].text = str(years["FY23"])
    row.cells[col_FY22].text = str(years["FY22"])

# ==============================================
# 6) Replace the placeholder with the SUMMARY text
#     (placeholder: [INSERT FINANCIAL PERFORMANCE SUMMARY])
# ==============================================
PLACEHOLDER = "[INSERT FINANCIAL PERFORMANCE SUMMARY]"

def set_paragraph_multiline(paragraph, text: str):
    # clear runs
    for run in paragraph.runs:
        run.text = ""
    # add lines with explicit line breaks
    lines = text.splitlines()
    if not lines:
        return
    paragraph.add_run(lines[0])
    for ln in lines[1:]:
        paragraph.add_run().add_break(WD_BREAK.LINE)
        paragraph.add_run(ln)

def replace_placeholder(document: Document, placeholder: str, new_text: str) -> bool:
    # search in paragraphs
    for p in document.paragraphs:
        if placeholder in p.text:
            set_paragraph_multiline(p, new_text)
            return True
    # search inside tables (cells contain their own paragraphs)
    for tbl in document.tables:
        for row in tbl.rows:
            for cell in row.cells:
                for p in cell.paragraphs:
                    if placeholder in p.text:
                        set_paragraph_multiline(p, new_text)
                        return True
    return False

if summary_text:
    ok = replace_placeholder(doc, PLACEHOLDER, summary_text)
    if not ok:
        print("WARNING: placeholder not found:", PLACEHOLDER)
else:
    print("NOTE: No summary text found in GPT output (no 'Summary / Interpretation' section).")

# ==============================================
# 7) Save
# ==============================================
out_path = "/Users/felipesilverio/Documents/GitHub/Azure-OnePager/CompanyProfile2.docx"
doc.save(out_path)
print(f"Updated document written to: {out_path}")

if not_found:
    print("WARNING — CSV metrics not matched to any row:")
    for m in not_found:
        print(" -", m)


Updated document written to: /Users/felipesilverio/Documents/GitHub/Azure-OnePager/CompanyProfile2.docx
WARNING — CSV metrics not matched to any row:
 - Revenue (Turnover)
 - Revenue growth % (yoy)
 - Cost of sales
 - Operating profit
 - Depreciation
 - Amortization
 - Net cash flow from operating activities
 - Net working capital (cash flow changes)
 - Cash flow from operating activities excl. working capital
 - Net cash flow from investing activities
 - Net cash flow from financing activities
 - Debt issuance (draw down of bank loans)
 - Share issuance
 - Leverage (Net Debt/EBITDA)


In [1]:
import io, re
import pandas as pd
from docx import Document
from docx.enum.text import WD_BREAK
from typing import Dict, List, Optional, Union

def insert_table_data_generic(
    gpt_output: str,
    doc_path: str,
    table_type: str,
    metric_mapping: Optional[Dict[str, str]] = None,
    doc: Optional[Document] = None
) -> Document:
    """
    Generic function to insert GPT-generated data into docx tables.
    
    Args:
        gpt_output: The GPT response containing CSV/Table data and summary
        doc_path: Path to the docx file
        table_type: Type of table - 'capital_structure', 'financial_performance', or 'key_stakeholders'
        metric_mapping: Optional dict to map CSV metric names to template row labels
                       Keys should be normalized (lowercase, no special chars)
        doc: Optional existing Document object. If None, will load from doc_path
    
    Returns:
        Updated Document object
    """
    
    # =========================
    # Helper functions
    # =========================
    def norm(s: str) -> str:
        """Normalize string for comparison (lowercase, alphanumeric only)"""
        return re.sub(r"[^a-z0-9]+", "", (s or "").lower())
    
    def keynorm(s: str) -> str:
        """Normalize string for dictionary keys"""
        return re.sub(r"[^a-z0-9]+", "", (s or "").lower())
    
    def tokens(s: str) -> set:
        return set(re.findall(r"[a-z0-9]+", (s or "").lower()))
    
    def jaccard(a: str, b: str) -> float:
        """Calculate Jaccard similarity between two strings"""
        ta, tb = tokens(a), tokens(b)
        if not ta or not tb:
            return 0.0
        inter = len(ta & tb)
        union = len(ta | tb)
        return inter / union if union else 0.0
    
    def set_cell_text(cell, text: str):
        """Set cell text preserving formatting"""
        if not cell.paragraphs:
            cell.add_paragraph(text)
            return
        p = cell.paragraphs[0]
        for run in p.runs:
            run.text = ""
        p.add_run(text)
    
    def set_paragraph_multiline(paragraph, text: str):
        """Replace a paragraph's text with multi-line content, preserving line breaks."""
        for run in paragraph.runs:
            run.text = ""
        lines = (text or "").splitlines()
        if not lines:
            return
        paragraph.add_run(lines[0])
        for ln in lines[1:]:
            r = paragraph.add_run()
            r.add_break(WD_BREAK.LINE)
            paragraph.add_run(ln)
    
    # =========================
    # 1) Extract CSV/Table + Summary from GPT output
    # =========================
    parts = gpt_output.split("\n\nSummary / Interpretation", 1)
    csv_block = parts[0].strip()
    
    # Find where the table/CSV starts
    start = csv_block.find("Metric,")
    if start == -1:
        # Try to find markdown table format
        start = csv_block.find("| Metric |")
        if start != -1:
            # Convert markdown table to CSV
            lines = csv_block[start:].split("\n")
            csv_lines = []
            for line in lines:
                line = line.strip()
                if line.startswith("|") and "---" not in line:
                    # Remove leading/trailing pipes and split
                    cells = [c.strip() for c in line.strip("|").split("|")]
                    csv_lines.append(",".join(f'"{c}"' if "," in c else c for c in cells))
                elif not line.startswith("|"):
                    break
            csv_block = "\n".join(csv_lines)
        else:
            raise ValueError("CSV/Table header 'Metric,' not found in model output.")
    else:
        csv_block = csv_block[start:]
    
    summary_text = ""
    if len(parts) > 1:
        summary_text = "Summary / Interpretation" + parts[1].rstrip()
    
    # =========================
    # 2) Parse CSV to DataFrame
    # =========================
    df = pd.read_csv(io.StringIO(csv_block))
    
    # Determine expected columns based on table type
    if table_type == "financial_performance":
        expected_cols = {"Metric", "FY24", "FY23", "FY22"}
        year_cols = ["FY24", "FY23", "FY22"]
    elif table_type == "capital_structure":
        expected_cols = {"Metric", "FY24"}
        year_cols = ["FY24"]
    elif table_type == "key_stakeholders":
        expected_cols = {"Metric", "Shareholders"}
        year_cols = ["Shareholders"]
    else:
        raise ValueError(f"Unknown table_type: {table_type}")
    
    if not expected_cols.issubset(df.columns):
        raise ValueError(f"{table_type} CSV columns missing. Expected {expected_cols}, Got: {list(df.columns)}")
    
    # Create dict: normalized metric name -> values
    if table_type in ["financial_performance", "capital_structure"]:
        csv_rows = {
            keynorm(str(df.at[i, "Metric"]).strip()): {
                col: str(df.at[i, col]) for col in year_cols
            }
            for i in range(len(df))
        }
    else:  # key_stakeholders
        csv_rows = {
            keynorm(str(df.at[i, "Metric"]).strip()): str(df.at[i, "Shareholders"]).strip()
            for i in range(len(df))
        }
    
    # =========================
    # 3) Open DOCX and find the target table
    # =========================
    if doc is None:
        doc = Document(doc_path)
    
    def find_table_by_type(document: Document, ttype: str):
        """Find table based on type"""
        if ttype == "financial_performance":
            # Look for table with FY24/FY23/FY22 headers
            for tbl in document.tables:
                if len(tbl.rows):
                    header = " ".join(c.text for c in tbl.rows[0].cells)
                    if all(x in norm(header) for x in ["fy24", "fy23", "fy22"]):
                        return tbl
            # Fallback: after "Financial Performance" heading
            return find_table_after_heading(document, "financialperformance")
        
        elif ttype == "capital_structure":
            # Look for table after "Capital Structure" heading
            return find_table_after_heading(document, "capitalstructure")
        
        elif ttype == "key_stakeholders":
            # Look for table with "Title" and "Occupants" columns
            for tbl in document.tables:
                if not tbl.rows:
                    continue
                header = [norm(c.text) for c in tbl.rows[0].cells]
                if "title" in header and "occupants" in header:
                    return tbl
            # Fallback: after "Key Stakeholders" heading
            return find_table_after_heading(document, "keystakeholders")
        
        return None
    
    def find_table_after_heading(document: Document, heading_normalized: str):
        """Find first table after a specific heading"""
        found_heading = False
        body = document._element.body
        for child in body.iterchildren():
            tag = child.tag.rsplit("}", 1)[-1]
            if tag == "p":
                p_text = "".join(
                    t.text for t in child.iter() if t.tag.rsplit("}", 1)[-1] == "t"
                ).strip()
                if norm(p_text) == heading_normalized:
                    found_heading = True
            elif tag == "tbl" and found_heading:
                from docx.table import Table
                return Table(child, document)
        return None
    
    table = find_table_by_type(doc, table_type)
    if table is None:
        raise RuntimeError(f"Could not locate the '{table_type}' table.")
    
    # =========================
    # 4) Populate the table
    # =========================
    
    # Detect column indices from header row
    header_row = table.rows[0]
    col_map = {}  # normalized header -> column index
    for idx, cell in enumerate(header_row.cells):
        col_map[norm(cell.text)] = idx
    
    # Build mapping for year columns
    year_col_indices = {}
    if table_type in ["financial_performance", "capital_structure"]:
        for year in year_cols:
            year_norm = norm(year)
            if year_norm in col_map:
                year_col_indices[year] = col_map[year_norm]
    else:  # key_stakeholders
        # Value column could be "Shareholders" or "Occupants"
        value_col_idx = col_map.get("shareholders") or col_map.get("occupants")
        if value_col_idx is None:
            value_col_idx = 1  # Default to second column
    
    # Populate data rows
    for row_idx in range(1, len(table.rows)):
        row = table.rows[row_idx]
        label_cell = row.cells[0]
        label_text = label_cell.text.strip()
        label_norm = keynorm(label_text)
        
        # Try direct match first
        matched_key = None
        if label_norm in csv_rows:
            matched_key = label_norm
        elif metric_mapping and label_norm in metric_mapping:
            # Use provided mapping
            mapped_key = keynorm(metric_mapping[label_norm])
            if mapped_key in csv_rows:
                matched_key = mapped_key
        else:
            # Try fuzzy matching with Jaccard similarity
            best_score = 0.0
            for csv_key in csv_rows.keys():
                score = jaccard(label_norm, csv_key)
                if score > best_score and score >= 0.6:  # Threshold
                    best_score = score
                    matched_key = csv_key
        
        # Populate cells if we found a match
        if matched_key:
            if table_type in ["financial_performance", "capital_structure"]:
                for year, col_idx in year_col_indices.items():
                    if col_idx < len(row.cells):
                        value = csv_rows[matched_key].get(year, "")
                        set_cell_text(row.cells[col_idx], str(value))
            else:  # key_stakeholders
                if value_col_idx < len(row.cells):
                    value = csv_rows[matched_key]
                    set_cell_text(row.cells[value_col_idx], str(value))
    
    # =========================
    # 5) Insert Summary below the table (if present)
    # =========================
    if summary_text:
        # Find the table in the document body and add summary after it
        table_elem = table._element
        parent = table_elem.getparent()
        table_idx = list(parent).index(table_elem)
        
        # Look for existing summary paragraph after the table
        summary_inserted = False
        for i in range(table_idx + 1, len(parent)):
            child = parent[i]
            tag = child.tag.rsplit("}", 1)[-1]
            if tag == "p":
                p_text = "".join(
                    t.text for t in child.iter() if t.tag.rsplit("}", 1)[-1] == "t"
                ).strip()
                if "summary" in norm(p_text) or "interpretation" in norm(p_text):
                    # Found summary section - update it
                    from docx.text.paragraph import Paragraph
                    para = Paragraph(child, doc)
                    set_paragraph_multiline(para, summary_text)
                    summary_inserted = True
                    break
            elif tag == "tbl":
                # Hit another table, stop looking
                break
        
        # If no existing summary found, add new paragraph
        if not summary_inserted:
            # Add paragraph after table
            new_para = doc.add_paragraph()
            set_paragraph_multiline(new_para, summary_text)
    
    return doc


# =========================
# Convenience wrapper functions
# =========================

def insert_capital_structure(gpt_output: str, doc_path: str = None, doc: Document = None) -> Document:
    """Insert capital structure data into docx table."""
    return insert_table_data_generic(
        gpt_output=gpt_output,
        doc_path=doc_path,
        table_type="capital_structure",
        doc=doc
    )


def insert_finance(gpt_output: str, doc_path: str = None, doc: Document = None,
                   metric_mapping: Optional[Dict[str, str]] = None) -> Document:
    """Insert financial performance data into docx table."""
    # Default metric mapping for financial performance
    if metric_mapping is None:
        metric_mapping = {
            keynorm("Revenue (Turnover)"): "Revenue",
            keynorm("Revenue growth % (yoy)"): "Revenue Growth",
            keynorm("Gross profit"): "Gross Profit",
            keynorm("Gross margin %"): "Gross Margin",
            keynorm("EBITDA"): "EBITDA",
            keynorm("EBITDA margin %"): "EBITDA Margin",
            keynorm("Adjusted EBITDA"): "Adjusted EBITDA",
            keynorm("Capex (tangible+intangible)"): "CAPEX",
            keynorm("CFADS"): "CFADS",
            keynorm("Net working capital change"): "Net Working Capital Change",
            keynorm("Total debt"): "Total Debt",
            keynorm("Net debt"): "Net Debt",
            keynorm("Leverage (Net Debt/EBITDA)"): "Leverage",
        }
    
    return insert_table_data_generic(
        gpt_output=gpt_output,
        doc_path=doc_path,
        table_type="financial_performance",
        metric_mapping=metric_mapping,
        doc=doc
    )


def insert_stakeholders(gpt_output: str, doc_path: str = None, doc: Document = None) -> Document:
    """Insert key stakeholders data into docx table."""
    return insert_table_data_generic(
        gpt_output=gpt_output,
        doc_path=doc_path,
        table_type="key_stakeholders",
        doc=doc
    )


# Helper to ensure keynorm is available
def keynorm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+", "", (s or "").lower())

In [2]:
insert_stakeholders
# =========================
# TEST: insert_stakeholders
# =========================

# Sample GPT output (this is what your agent would return)
stakeholders_test_output = """
Metric,Shareholders
"Shareholders","Seaport Holdings Ltd (Immediate Parent); Ultimate Parent: Private Equity Group XYZ"
"Management","CEO: John Smith; CFO: Jane Doe; Chairman: Robert Brown"
"Lenders","Bank of America (Term Loan A); JPMorgan Chase (RCF)"
"Auditors","Grant Thornton UK LLP"
"Advisors","Financial Advisor: Goldman Sachs; Legal: Clifford Chance"

Summary / Interpretation
- The company is owned by Private Equity Group XYZ through Seaport Holdings Ltd.
- Management team includes experienced executives with John Smith as CEO.
- Primary lenders are Bank of America and JPMorgan Chase.
- Grant Thornton serves as the statutory auditor.
- Goldman Sachs and Clifford Chance provide financial and legal advisory services.

Sources
- [#1] Annual Report FY24, Directors' Report, p.12
- [#2] Annual Report FY24, Notes to Financial Statements
"""

# Test the function
print("Testing insert_stakeholders...")
try:
    # Load your template document
    doc_path = "/Users/felipesilverio/Documents/GitHub/Azure-OnePager/CompanyProfile (1).docx"
    
    # Call the function
    doc = insert_stakeholders(stakeholders_test_output, doc_path=doc_path)
    
    # Save to a test output file
    output_path = "/Users/felipesilverio/Documents/GitHub/Azure-OnePager/test_stakeholders.docx"
    doc.save(output_path)
    
    print(f"✓ Success! Open the file to check: {output_path}")
except Exception as e:
    print(f"✗ Error: {e}")
    import traceback
    traceback.print_exc()


Testing insert_stakeholders...
✓ Success! Open the file to check: /Users/felipesilverio/Documents/GitHub/Azure-OnePager/test_stakeholders.docx
